In [ ]:
import pandas as pd         # 导入Pandas库，用于数据处理
import numpy as np          # 导入NumPy库，用于数值计算
import os
import matplotlib.pyplot as plt

# 导入模型选择模块中的train_test_split，用于拆分数据集
from sklearn.model_selection import train_test_split

# 导入预处理模块中的StandardScaler 和OneHotEncoder，用于数据标准化和独热编码
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 导入逻辑回归模型
from sklearn.linear_model import LogisticRegression

# 导入随机森林分类器
from sklearn.ensemble import RandomForestClassifier

# 导入多层感知器分类器
from sklearn.neural_network import MLPClassifier

# 导入支持向量机分类器
from sklearn.svm import SVC

# 导入XGBoost库
# !pip install xgboost -i https://mirrors.aliyun.com/pypi/simple/
import xgboost as xgb

#导入LightGBM库
import lightgbm as lgb

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble  import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier     # pip install catboost
from sklearn.ensemble import GradientBoostingClassifier
import statsmodels.api as sm                # pip install statsmodels
from pytorch_tabnet.tab_model import TabNetClassifier

import shap
# plt.rcParams['font.sans-serif'] = ['SimSun'] #宋体
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei']

In [ ]:
# 动态基线特征提起取和缺失值过滤相关函数
import pandas as pd
import numpy as np
import os
from tqdm import tqdm  # 用于显示进度条
import matplotlib.pyplot as plt
from collections import defaultdict
import json
from datetime import datetime

# ==============================================================
# 0. 数据监控类（动态基线特征提取函数定义——补充功能）
# ==============================================================
class VariableMonitor:
    """
    监控特定变量在数据提取过程中的空数据情况。
    
    功能：
    1. 追踪指定变量何时为空
    2. 记录在指定时间区间内未找到有效数据的患者ID
    3. 保存患者ID以供后续手动检查
    
    用法示例：
        monitor = VariableMonitor(variables_to_monitor=['CBC004', 'CBC001'])
        # 在数据提取过程中调用 record_patient_data 方法
        # 最后调用 get_empty_patient_ids() 或 save_report() 获取结果
    """
    
    def __init__(self, variables_to_monitor=None, time_window=(-15, 0)):
        """
        初始化监控器
        
        参数:
            variables_to_monitor: 要监控的变量列表，例如 ['CBC004', 'CBC001']
            time_window: 监控的时间窗口，默认 (-15, 0)
        """
        self.variables_to_monitor = variables_to_monitor or []
        self.time_window = time_window
        
        # 记录每个变量的空数据患者ID
        # 结构: {variable_name: set(patient_ids)}
        self.empty_data_patients = {var: set() for var in self.variables_to_monitor}
        
        # 记录详细信息（可选）
        # 结构: {variable_name: {patient_id: {'reason': str, 'time_range': tuple}}}
        self.detailed_info = {var: {} for var in self.variables_to_monitor}
        
        # 统计信息
        self.total_patients_processed = 0
        
    def record_patient_data(self, patient_id, variable_name, has_data, time_range=None, reason=None):
        """
        记录患者的数据状态
        
        参数:
            patient_id: 患者ID
            variable_name: 变量名称
            has_data: 是否有有效数据 (bool)
            time_range: 实际数据的时间范围 (可选)
            reason: 为空的原因 (可选)，例如 'no_file', 'no_data_in_window', 'variable_missing'
        """
        if variable_name not in self.variables_to_monitor:
            return
        
        if not has_data:
            self.empty_data_patients[variable_name].add(patient_id)
            self.detailed_info[variable_name][patient_id] = {
                'reason': reason or 'unknown',
                'time_range': time_range,
                'timestamp': datetime.now().isoformat()
            }
    
    def get_empty_patient_ids(self, variable_name=None):
        """
        获取指定变量（或所有变量）的空数据患者ID集合
        
        参数:
            variable_name: 变量名称，如果为 None，返回所有变量的结果
            
        返回:
            如果指定变量名，返回该变量的患者ID集合
            如果未指定，返回字典 {variable_name: set(patient_ids)}
        """
        if variable_name:
            return self.empty_data_patients.get(variable_name, set())
        return self.empty_data_patients
    
    def get_summary(self):
        """
        获取监控摘要信息
        
        返回:
            dict: 包含各变量的统计信息
        """
        summary = {}
        for var in self.variables_to_monitor:
            empty_count = len(self.empty_data_patients[var])
            summary[var] = {
                'empty_patient_count': empty_count,
                'empty_patient_ids': sorted(list(self.empty_data_patients[var])),
                'time_window': self.time_window
            }
        
        summary['total_patients_processed'] = self.total_patients_processed
        return summary

    def save_report(self, output_path='monitor_report.json', include_details=True):
        """
        保存监控报告到文件
        
        参数:
            output_path: 输出文件路径
            include_details: 是否包含详细信息
        """
        report = {
            'monitoring_config': {
                'variables': self.variables_to_monitor,
                'time_window': self.time_window,
                'total_patients': int(self.total_patients_processed),  # 转换为 int
                'generated_at': datetime.now().isoformat()
            },
            'summary': self._convert_to_json_serializable(self.get_summary())  # 递归转换
        }
        
        if include_details:
            report['detailed_info'] = self._convert_to_json_serializable({
                var: {pid: info for pid, info in patient_info.items()}
                for var, patient_info in self.detailed_info.items()
            })
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(report, f, indent=2, ensure_ascii=False)
        
        print(f"\n监控报告已保存至: {output_path}")
        return output_path

    def _convert_to_json_serializable(self, obj):
            """
            递归转换 NumPy/Pandas 类型为 JSON 可序列化的 Python 原生类型
            递归处理字典的 Keys 和 Values
            """
            import numpy as np
            
            if isinstance(obj, dict):
                # 遍历字典时，同时通过 int() 转换 Key，防止 Keys 是 int64 类型
                new_dict = {}
                for key, value in obj.items():
                    # 如果 Key 是 numpy 整数类型，强转为 Python int
                    if isinstance(key, (np.integer, np.int64, np.int32)):
                        new_key = int(key)
                    else:
                        new_key = key
                    new_dict[new_key] = self._convert_to_json_serializable(value)
                return new_dict
                
            elif isinstance(obj, list):
                return [self._convert_to_json_serializable(item) for item in obj]
            elif isinstance(obj, set):
                return sorted([self._convert_to_json_serializable(item) for item in obj])
            elif isinstance(obj, (np.integer, np.int64, np.int32)):
                return int(obj)
            elif isinstance(obj, (np.floating, np.float64, np.float32)):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            else:
                return obj
        
    def print_summary(self):
        """打印监控摘要到控制台"""
        print("\n" + "="*60)
        print("变量监控摘要报告")
        print("="*60)
        print(f"监控时间窗口: {self.time_window}")
        print(f"处理患者总数: {self.total_patients_processed}")
        print("-"*60)
        
        for var in self.variables_to_monitor:
            empty_count = len(self.empty_data_patients[var])
            print(f"\n变量: {var}")
            print(f"  空数据患者数: {empty_count}")
            if empty_count > 0:
                print(f"  患者ID列表: {sorted(list(self.empty_data_patients[var]))[:10]}" + 
                      (f" ... (共{empty_count}个)" if empty_count > 10 else ""))
        
        print("="*60 + "\n")


# ==============================================================
# 1.动态基线特征提取函数定义（修复增强版）
# ==============================================================
def extract_baseline_features(patient_ids, dynamic_dir, time_col='day', window_start=-15, cutoff_day=0, monitor=None):
    """
    从动态文件中提取 [window_start, cutoff_day] 区间内的最后一次有效观测值作为基线特征。
    
    参数:
        patient_ids: 患者 ID 列表
        dynamic_dir: 动态数据文件夹路径
        time_col: 时间列名，支持大小写模糊匹配，默认为 'day'
        window_start: 时间窗口起始天数 (默认 -15)
        cutoff_day: 时间窗口结束天数 (默认 0)
        monitor: 监控器对象 (可选)
        逻辑说明：Sort -> Ffill -> Take Last
    返回:
        df_features: 提取后的特征 DataFrame
        missing_pids: 在指定窗口内完全无数据的患者ID列表
    """
    extracted_features = []
    missing_pids = [] # 用于存储缺失数据的患者ID
    
    print(f"正在从 {len(patient_ids)} 个动态文件中提取基线特征 (Day [{window_start}, {cutoff_day}])...")
    
    # 更新监控器的患者总数
    if monitor:
        monitor.total_patients_processed = len(patient_ids)
    
    for pid in tqdm(patient_ids):
        file_path = os.path.join(dynamic_dir, f"{pid}.csv")
        
        # 初始化字典，包含 ID
        patient_data = {'ID': pid}
        
        if os.path.exists(file_path):
            try:
                # 读取动态数据
                df_dyn = pd.read_csv(file_path)
                
                # 1. 健壮性修复：处理时间列名大小写不一致问题
                actual_time_col = None
                if time_col in df_dyn.columns:
                    actual_time_col = time_col
                else:
                    # 尝试查找忽略大小写的匹配
                    col_map = {c.lower(): c for c in df_dyn.columns}
                    if time_col.lower() in col_map:
                        actual_time_col = col_map[time_col.lower()]
                
                if actual_time_col:
                    # 2. 健壮性修复：强制转换时间列为数值型，处理脏数据
                    df_dyn[actual_time_col] = pd.to_numeric(df_dyn[actual_time_col], errors='coerce')
                    df_dyn = df_dyn.dropna(subset=[actual_time_col]) # 删除时间解析失败的行

                    # 3. 筛选：严格保留 [window_start, cutoff_day] 区间的数据
                    mask = (df_dyn[actual_time_col] >= window_start) & (df_dyn[actual_time_col] <= cutoff_day)
                    df_baseline_window = df_dyn[mask].copy()
                    
                    if not df_baseline_window.empty:
                        # 4. 排序：按时间升序排列 (Day -15 -> Day 0)
                        df_sorted = df_baseline_window.sort_values(by=actual_time_col, ascending=True)
                        
                        # ==========================================================
                        # 核心修复逻辑
                        # ==========================================================
                        # 解决"存在数据但提取为空"的问题,原代码仅取最后一行(iloc[-1])，会导致之前行存在的有效数据（稀疏数据）被遗漏.
                        # 问题：如果 Day -5 有值，Day -1 无值(NaN)，只取最后一行(Day -1)会导致 NaN。
                        # 修复：ffill() 会将 Day -5 的有效值"携带"到 Day -1。
                        # 这样最后一行就包含了截至该时刻所有变量的最新已知值。
                        df_filled = df_sorted.ffill() 
                        last_observation = df_filled.iloc[-1]
                        
                        # 5. 转换：提取特征
                        for col in last_observation.index:
                            # 排除 ID, 时间列 和 可能的索引列
                            if col != actual_time_col and col != 'ID' and not col.startswith('Unnamed'): 
                                patient_data[f"baseline_{col}"] = last_observation[col]
                        
                        # 记录Gap Days (评估数据新鲜度)
                        patient_data['baseline_gap_days'] = cutoff_day - last_observation[actual_time_col]
                        
                        # 【监控功能】检查监控的变量是否提取到了数据
                        if monitor:
                            time_range = (df_baseline_window[actual_time_col].min(), 
                                        df_baseline_window[actual_time_col].max())
                            for var in monitor.variables_to_monitor:
                                # 检查该变量是否存在且有有效值
                                baseline_var_name = f"baseline_{var}"
                                has_data = False
                                if baseline_var_name in patient_data:
                                    has_data = pd.notna(patient_data[baseline_var_name])
                          
                                monitor.record_patient_data(
                                    patient_id=pid,
                                    variable_name=var,
                                    has_data=has_data,
                                    time_range=time_range,
                                    reason='value_is_nan' if not has_data else None
                                )

                    else:
                        # 修复要求 1: 数据为空时的异常处理
                        # 窗口内没有数据
                        print(f"⚠️ 警告: 患者 {pid} 在窗口 [{window_start}, {cutoff_day}] 内无有效数据")
                        missing_pids.append(pid)
                        
                        # 【监控功能】时间窗口内没有数据
                        if monitor:
                            for var in monitor.variables_to_monitor:
                                monitor.record_patient_data(
                                    patient_id=pid,
                                    variable_name=var,
                                    has_data=False,
                                    time_range=None,
                                    reason='no_data_in_time_window'
                                )
                else:
                    print(f"❌ 错误: 文件 {pid}.csv 中未找到时间列 '{time_col}'")
                    missing_pids.append(pid)

            except Exception as e:
                print(f"❌ 处理 ID {pid} 时发生异常: {e}")
                missing_pids.append(pid)
        else:
            # 文件不存在
            # print(f"文件不存在: {file_path}") # 可选：减少刷屏
            if monitor:
                for var in monitor.variables_to_monitor:
                    monitor.record_patient_data(
                        patient_id=pid,
                        variable_name=var,
                        has_data=False,
                        time_range=None,
                        reason='patient_file_not_found'
                    )
        
        extracted_features.append(patient_data)
    
    # 转换为 DataFrame
    df_features = pd.DataFrame(extracted_features)
    print(f"\n提取完成。共 {len(missing_pids)} 位患者基线数据完全缺失。")
    # if len(missing_pids) > 0:     # 可选：减少刷屏
    #     print(f"缺失数据患者ID示例: {missing_pids[:10]}")
    return df_features



# ==============================================================
# 2. 动态特征缺失值过滤函数定义
# ==============================================================
# 识别动态特征：自动识别以 baseline_ 开头的所有列
# 缺失值统计：计算每个动态特征的缺失数量和百分比
# 阈值过滤：删除缺失值 > 20% 的列
# 1.生成报告：
# - 缺失值统计 CSV 文件
# - 可视化缺失值分布图
# 2.新增 drop_high_missing 参数：
# - True（默认）：删除超过阈值的列（原有行为）
# - False：仅分析并报告，不删除任何列
# 3.增强的统计信息：
# - cols_above_threshold：记录超过阈值的列数
# - suggested_drop_cols：建议删除的列（无论是否实际删除）
# - drop_mode_enabled：记录当前模式
def filter_baseline_features_by_missing(df, missing_threshold=20.0, 
                                        output_dir='/home/phl/PHL/Car-T/model_v1/output/dynamic_feature_missing',
                                        verbose=True,
                                        drop_high_missing=True
                                        ):
    """
    过滤动态特征中缺失值超过阈值的列
    
    参数:
        df: DataFrame, 包含动态特征的数据框
        missing_threshold: float, 缺失值百分比阈值（默认20%）
        output_dir: str, 输出目录路径
        verbose: bool, 是否打印详细信息
        drop_high_missing: bool, 是否删除高缺失率列（True=删除，False=仅分析）
    
    返回:
        DataFrame: 过滤后的数据框（如果drop_high_missing=False，则返回原始数据框）
        dict: 包含统计信息的字典
    """
    if verbose:
        print("=" * 70)
        print(f"动态特征缺失值{'过滤' if drop_high_missing else '分析'}（阈值: {missing_threshold}%）")
        print("=" * 70)
    
    # 1. 自动识别以 baseline_ 开头的所有动态特征列
    baseline_cols = [col for col in df.columns if col.startswith('baseline_')]
    
    if verbose:
        print(f"\n提取到的动态特征列数量: {len(baseline_cols)}")
    
    if len(baseline_cols) == 0:
        print("⚠️ 警告：未找到任何动态特征列（baseline_*）")
        return df, {'baseline_cols': 0, 'cols_dropped': 0, 'cols_kept': 0}
    
    # 2. 计算每个动态特征的缺失数量和百分比
    missing_stats = {}
    for col in baseline_cols:
        missing_count = df[col].isna().sum()
        missing_pct = (missing_count / len(df)) * 100
        missing_stats[col] = {
            'missing_count': missing_count,
            'missing_pct': missing_pct
        }
    
    # 转换为 DataFrame
    df_missing_stats = pd.DataFrame(missing_stats).T
    df_missing_stats = df_missing_stats.sort_values('missing_pct', ascending=False)
    
    if verbose:
        print(f"\n动态特征缺失值统计（前10个）:")
        print(df_missing_stats.head(10).to_string())
    
    # 3. 应用阈值过滤，删除缺失值 > 20% 的列
    cols_to_drop = [col for col, stats in missing_stats.items() 
                    if stats['missing_pct'] > missing_threshold]
    cols_to_keep = [col for col, stats in missing_stats.items() 
                    if stats['missing_pct'] <= missing_threshold]
    
    if verbose:
        print(f"\n缺失值{'过滤' if drop_high_missing else '分析'}结果:")
        print(f"  超过阈值的列数: {len(cols_to_drop)}")
        print(f"  低于阈值的列数: {len(cols_to_keep)}")
    
    # 4. 根据参数决定是否删除列
    df_filtered = df.copy()
    if drop_high_missing and cols_to_drop:
        if verbose:
            print(f"\n正在删除 {len(cols_to_drop)} 个高缺失率动态特征...")
            print(f"删除的列示例（前5个）: {cols_to_drop[:5]}")
        df_filtered = df_filtered.drop(columns=cols_to_drop)
    elif not drop_high_missing:
        if verbose:
            print(f"\n⚠️ 仅分析模式：未删除任何列")
            if cols_to_drop:
                print(f"建议删除的列（超过{missing_threshold}%缺失率）:")
                for col in cols_to_drop[:10]:  # 显示前10个
                    pct = missing_stats[col]['missing_pct']
                    print(f"  - {col}: {pct:.2f}%")
    else:
        if verbose:
            print("\n✅ 所有动态特征的缺失值比例均低于阈值，无需删除")
    
    # 5. 生成报告：缺失值统计 CSV 文件
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    report_path = os.path.join(output_dir, 'baseline_features_missing_report.csv')
    df_missing_stats.to_csv(report_path)
    if verbose:
        print(f"\n缺失值统计报告已保存至: {report_path}")
    
    # 6. 可视化：缺失值分布图（水平条形图）
    if len(df_missing_stats) > 0:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        plot_data = df_missing_stats.head(30).sort_values('missing_pct', ascending=True)
        colors = ['#E74C3C' if pct > missing_threshold else '#2ECC71' 
                  for pct in plot_data['missing_pct']]
        
        ax.barh(range(len(plot_data)), plot_data['missing_pct'], color=colors, alpha=0.7)
        ax.set_yticks(range(len(plot_data)))
        ax.set_yticklabels(plot_data.index, fontsize=8)
        ax.axvline(x=missing_threshold, color='red', linestyle='--', linewidth=2, 
                   label=f'阈值 ({missing_threshold}%)')
        ax.set_xlabel('缺失值百分比 (%)', fontsize=12, fontweight='bold')
        
        # 根据模式调整标题
        title = f'动态特征缺失值分布 (Top 30) - {"删除模式" if drop_high_missing else "仅分析"}'
        ax.set_title(title, fontsize=14, fontweight='bold')
        ax.legend(fontsize=10)
        ax.grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plot_path = os.path.join(output_dir, 'baseline_features_missing_plot.png')
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        if verbose:
            print(f"缺失值分布图已保存至: {plot_path}")
        plt.close()
    
    # 7. 准备返回的统计信息
    stats_summary = {
        'baseline_cols': len(baseline_cols),
        'cols_dropped': len(cols_to_drop) if drop_high_missing else 0,
        'cols_kept': len(cols_to_keep),
        'cols_above_threshold': len(cols_to_drop),  # 记录超过阈值的列数
        'original_shape': df.shape,
        'filtered_shape': df_filtered.shape,
        'missing_stats_df': df_missing_stats,
        'dropped_cols': cols_to_drop if drop_high_missing else [],
        'suggested_drop_cols': cols_to_drop,  # 建议删除的列（无论是否真正删除）
        'drop_mode_enabled': drop_high_missing  # 记录是否启用删除模式
    }
    
    if verbose:
        print(f"\n处理后数据形状: {df_filtered.shape}")
        remaining_baseline = sum(1 for col in df_filtered.columns if col.startswith('baseline_'))
        print(f"剩余动态特征列数: {remaining_baseline}")
        
        if not drop_high_missing and cols_to_drop:
            print(f"\n💡 提示: 当前为仅分析模式，未删除任何列")
            print(f"   如需删除高缺失率列，请设置 drop_high_missing=True")
    
    return df_filtered, stats_summary

# ==============================================================
# 3.静态特征缺失值过滤函数定义
# ==============================================================
# 专门针对静态特征：自动排除 baseline_ 开头的动态特征和指定的排除列
# 灵活的排除列表：通过 exclude_cols 参数自定义需要排除的列
# 与动态特征一致的逻辑：相同的参数、返回值和可视化风格
def filter_static_features_by_missing(df, missing_threshold=20.0, 
                                      output_dir='/home/phl/PHL/Car-T/model_v1/output/static_feature_missing',
                                      verbose=True,
                                      drop_high_missing=True,
                                      exclude_cols=None):
    """
    过滤静态特征中缺失值超过阈值的列
    
    参数:
        df: DataFrame, 包含静态特征的数据框
        missing_threshold: float, 缺失值百分比阈值（默认20%）
        output_dir: str, 输出目录路径
        verbose: bool, 是否打印详细信息
        drop_high_missing: bool, 是否删除高缺失率列（True=删除，False=仅分析）
        exclude_cols: list, 需要排除分析的列（如ID、标签列等）
    
    返回:
        DataFrame: 过滤后的数据框（如果drop_high_missing=False，则返回原始数据框）
        dict: 包含统计信息的字典
    """
    if verbose:
        print("=" * 70)
        print(f"静态特征缺失值{'过滤' if drop_high_missing else '分析'}（阈值: {missing_threshold}%）")
        print("=" * 70)
    
    # 1. 确定要分析的列
    # 默认排除的列：ID、目标变量
    default_exclude = ['ID', 'Infection', 'CRS', 'ICANS', 'E_ICAHT', 'L_ICAHT']
    if exclude_cols is None:
        exclude_cols = default_exclude
    else:
        exclude_cols = list(set(default_exclude + exclude_cols))
    
    # 提取静态特征列（排除baseline_开头的动态特征和需要排除的列）
    static_cols = [col for col in df.columns 
                   if not col.startswith('baseline_') and col not in exclude_cols]
    
    if verbose:
        print(f"\n提取到的静态特征列数量: {len(static_cols)}")
        print(f"排除的列: {exclude_cols}")
    
    if len(static_cols) == 0:
        print("⚠️ 警告：未找到任何静态特征列")
        return df, {'static_cols': 0, 'cols_dropped': 0, 'cols_kept': 0}
    
    # 2. 计算缺失值统计
    missing_stats = {}
    for col in static_cols:
        missing_count = df[col].isna().sum()
        missing_pct = (missing_count / len(df)) * 100
        missing_stats[col] = {
            'missing_count': missing_count,
            'missing_pct': missing_pct
        }
    
    # 转换为 DataFrame
    df_missing_stats = pd.DataFrame(missing_stats).T
    df_missing_stats = df_missing_stats.sort_values('missing_pct', ascending=False)
    
    if verbose:
        print(f"\n静态特征缺失值统计（前10个）:")
        print(df_missing_stats.head(10).to_string())
    
    # 3. 应用阈值过滤
    cols_to_drop = [col for col, stats in missing_stats.items() 
                    if stats['missing_pct'] > missing_threshold]
    cols_to_keep = [col for col, stats in missing_stats.items() 
                    if stats['missing_pct'] <= missing_threshold]
    
    if verbose:
        print(f"\n缺失值{'过滤' if drop_high_missing else '分析'}结果:")
        print(f"  超过阈值的列数: {len(cols_to_drop)}")
        print(f"  低于阈值的列数: {len(cols_to_keep)}")
    
    # 4. 根据参数决定是否删除列
    df_filtered = df.copy()
    if drop_high_missing and cols_to_drop:
        if verbose:
            print(f"\n正在删除 {len(cols_to_drop)} 个高缺失率静态特征...")
            print(f"删除的列示例（前5个）: {cols_to_drop[:5]}")
        df_filtered = df_filtered.drop(columns=cols_to_drop)
    elif not drop_high_missing:
        if verbose:
            print(f"\n⚠️ 仅分析模式：未删除任何列")
            if cols_to_drop:
                print(f"建议删除的列（超过{missing_threshold}%缺失率）:")
                for col in cols_to_drop[:10]:
                    pct = missing_stats[col]['missing_pct']
                    print(f"  - {col}: {pct:.2f}%")
    else:
        if verbose:
            print("\n✅ 所有静态特征的缺失值比例均低于阈值，无需删除")
    
    # 5. 保存统计报告
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    report_path = os.path.join(output_dir, 'static_features_missing_report.csv')
    df_missing_stats.to_csv(report_path)
    if verbose:
        print(f"\n缺失值统计报告已保存至: {report_path}")
    
    # 6. 可视化
    if len(df_missing_stats) > 0:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # 取前30个特征进行展示（如果静态特征少于30个则全部展示）
        n_features_to_plot = min(30, len(df_missing_stats))
        plot_data = df_missing_stats.head(n_features_to_plot).sort_values('missing_pct', ascending=True)
        
        colors = ['#E74C3C' if pct > missing_threshold else '#2ECC71' 
                  for pct in plot_data['missing_pct']]
        
        ax.barh(range(len(plot_data)), plot_data['missing_pct'], color=colors, alpha=0.7)
        ax.set_yticks(range(len(plot_data)))
        ax.set_yticklabels(plot_data.index, fontsize=8)
        ax.axvline(x=missing_threshold, color='red', linestyle='--', linewidth=2, 
                   label=f'阈值 ({missing_threshold}%)')
        ax.set_xlabel('缺失值百分比 (%)', fontsize=12, fontweight='bold')
        
        # 根据模式调整标题
        title = f'静态特征缺失值分布 (Top {n_features_to_plot}) - {"删除模式" if drop_high_missing else "仅分析"}'
        ax.set_title(title, fontsize=14, fontweight='bold')
        ax.legend(fontsize=10)
        ax.grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plot_path = os.path.join(output_dir, 'static_features_missing_plot.png')
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        if verbose:
            print(f"缺失值分布图已保存至: {plot_path}")
        plt.close()
    
    # 7. 准备返回的统计信息
    stats_summary = {
        'static_cols': len(static_cols),
        'cols_dropped': len(cols_to_drop) if drop_high_missing else 0,
        'cols_kept': len(cols_to_keep),
        'cols_above_threshold': len(cols_to_drop),
        'original_shape': df.shape,
        'filtered_shape': df_filtered.shape,
        'missing_stats_df': df_missing_stats,
        'dropped_cols': cols_to_drop if drop_high_missing else [],
        'suggested_drop_cols': cols_to_drop,
        'drop_mode_enabled': drop_high_missing
    }
    
    if verbose:
        print(f"\n处理后数据形状: {df_filtered.shape}")
        remaining_static = sum(1 for col in df_filtered.columns 
                              if not col.startswith('baseline_') and col not in exclude_cols)
        print(f"剩余静态特征列数: {remaining_static}")
        
        if not drop_high_missing and cols_to_drop:
            print(f"\n💡 提示: 当前为仅分析模式，未删除任何列")
            print(f"   如需删除高缺失率列，请设置 drop_high_missing=True")
    
    return df_filtered, stats_summary

In [ ]:
# =============================================================================
# 通用特征删除函数
# =============================================================================
def drop_features_from_datasets(X_train_df, X_test_df, columns_to_drop=None, 
                                 feature_names=None, verbose=True, features_to_keep=None):
    """
    从训练集和测试集中同步删除指定的特征列
    
    参数:
        X_train_df: pd.DataFrame
            训练集DataFrame
        X_test_df: pd.DataFrame
            测试集DataFrame
        columns_to_drop: list, optional
            需要删除的特征列名列表
        feature_names: list, optional
            当前的特征名列表,如果提供则会同步更新
        verbose: bool, default=True
            是否打印详细日志
        features_to_keep: list, optional
            需要保留的特征列名列表(hold列表)。如果提供，将保留这些列并删除其他所有列。
    
    返回:
        tuple: (X_train_filtered, X_test_filtered, feature_names_updated, stats_dict)
            - X_train_filtered: 删除特征后的训练集
            - X_test_filtered: 删除特征后的测试集
            - feature_names_updated: 更新后的特征名列表
            - stats_dict: 删除操作的统计信息字典
    
    异常:
        ValueError: 当输入数据不合法时抛出
    """
    import pandas as pd
    import numpy as np
    
    # ===================== 步骤1: 输入验证 =====================
    if not isinstance(X_train_df, pd.DataFrame):
        raise ValueError("X_train_df 必须是 pandas DataFrame")
    if not isinstance(X_test_df, pd.DataFrame):
        raise ValueError("X_test_df 必须是 pandas DataFrame")
        
    if columns_to_drop is None and features_to_keep is None:
        raise ValueError("必须提供 columns_to_drop 或 features_to_keep 其中之一")

    if columns_to_drop is not None and not isinstance(columns_to_drop, (list, tuple, np.ndarray)):
        raise ValueError("columns_to_drop 必须是列表或数组")
    
    if features_to_keep is not None and not isinstance(features_to_keep, (list, tuple, np.ndarray)):
        raise ValueError("features_to_keep 必须是列表或数组")
    
    # ===================== 特殊逻辑: 处理 features_to_keep =====================
    if features_to_keep is not None:
        features_to_keep = list(features_to_keep)
        if verbose:
            print("=" * 80)
            print("特征保留操作 (Hold Mode)")
            print("=" * 80)
            print(f"\n请求保留的特征数 (Hold List): {len(features_to_keep)}")
            
        # 1. 检查保留列表中的列是否存在
        missing_in_train = [col for col in features_to_keep if col not in X_train_df.columns]
        missing_in_test = [col for col in features_to_keep if col not in X_test_df.columns]
        
        if verbose and (missing_in_train or missing_in_test):
            print(f"\n保留列验证详情:")
            if missing_in_train:
                print(f"  ⚠️ {len(missing_in_train)} 个保留列在训练集中缺失")
                for col in missing_in_train[:5]: print(f"    - {col}")
            if missing_in_test:
                print(f"  ⚠️ {len(missing_in_test)} 个保留列在测试集中缺失")
        
        # 2. 转换逻辑：除了保留列之外的所有列都需要被删除
        # 以训练集为基准计算要删除的列
        all_train_cols = X_train_df.columns.tolist()
        columns_to_drop = [col for col in all_train_cols if col not in features_to_keep]
        
        if verbose:
            print(f"\n计算结果:")
            print(f"  原特征总数: {len(all_train_cols)}")
            print(f"  将要保留的有效特征数: {len([c for c in all_train_cols if c in features_to_keep])}")
            print(f"  将要删除的特征数: {len(columns_to_drop)}")
            
    else:
        # ===================== 常规逻辑: 使用 columns_to_drop =====================
        # 转换为列表
        columns_to_drop = list(columns_to_drop)
        
        if verbose:
            print("=" * 80)
            print("特征删除操作")
            print("=" * 80)
            print(f"\n请求删除的特征数: {len(columns_to_drop)}")
    
    # ===================== 步骤2: 验证列是否存在 =====================
    # 检查哪些列存在于训练集中
    cols_in_train = [col for col in columns_to_drop if col in X_train_df.columns]
    cols_not_in_train = [col for col in columns_to_drop if col not in X_train_df.columns]
    
    # 检查哪些列存在于测试集中
    cols_in_test = [col for col in columns_to_drop if col in X_test_df.columns]
    cols_not_in_test = [col for col in columns_to_drop if col not in X_test_df.columns]
    
    # 找出在两个数据集中都存在的列
    cols_to_drop_valid = list(set(cols_in_train) & set(cols_in_test))
    
    # 找出仅在一个数据集中存在的列(可能的数据不一致)
    cols_only_in_train = list(set(cols_in_train) - set(cols_in_test))
    cols_only_in_test = list(set(cols_in_test) - set(cols_in_train))
    
    if verbose:
        print(f"\n列验证结果:")
        print(f"  ✓ 在两个数据集中都存在: {len(cols_to_drop_valid)} 列")
        if cols_not_in_train or cols_not_in_test:
            print(f"  ⚠ 在训练集中不存在: {len(cols_not_in_train)} 列")
            print(f"  ⚠ 在测试集中不存在: {len(cols_not_in_test)} 列")
        if cols_only_in_train:
            print(f"  ⚠ 仅在训练集中存在: {len(cols_only_in_train)} 列")
        if cols_only_in_test:
            print(f"  ⚠ 仅在测试集中存在: {len(cols_only_in_test)} 列")
        
        # 显示未找到的列示例
        if cols_not_in_train:
            print(f"\n  未在训练集中找到的列示例(前5个):")
            for col in cols_not_in_train[:5]:
                print(f"    - {col}")
        
        # 显示数据不一致的警告
        if cols_only_in_train or cols_only_in_test:
            print(f"\n  ⚠️ 警告: 训练集和测试集的列结构不一致!")
            if cols_only_in_train:
                print(f"    仅在训练集中存在的列(前3个): {cols_only_in_train[:3]}")
            if cols_only_in_test:
                print(f"    仅在测试集中存在的列(前3个): {cols_only_in_test[:3]}")
    
    # ===================== 步骤3: 记录删除前的状态 =====================
    original_train_shape = X_train_df.shape
    original_test_shape = X_test_df.shape
    original_n_features = X_train_df.shape[1]
    
    if verbose:
        print(f"\n删除前:")
        print(f"  训练集形状: {original_train_shape}")
        print(f"  测试集形状: {original_test_shape}")
        print(f"  特征数量: {original_n_features}")
    
    # ===================== 步骤4: 执行删除操作 =====================
    if len(cols_to_drop_valid) == 0:
        if verbose:
            print("\n⚠️ 没有有效的列可以删除!")
        X_train_filtered = X_train_df.copy()
        X_test_filtered = X_test_df.copy()
    else:
        if verbose:
            print(f"\n正在删除 {len(cols_to_drop_valid)} 个特征...")
            print(f"  删除的列示例(前10个):")
            for col in cols_to_drop_valid[:10]:
                print(f"    - {col}")
            if len(cols_to_drop_valid) > 10:
                print(f"    ... 还有 {len(cols_to_drop_valid) - 10} 列未显示")
        
        # 从训练集中删除
        X_train_filtered = X_train_df.drop(columns=cols_to_drop_valid, errors='ignore')
        
        # 从测试集中删除
        X_test_filtered = X_test_df.drop(columns=cols_to_drop_valid, errors='ignore')
    
    # ===================== 步骤5: 更新特征名列表 =====================
    if feature_names is not None:
        # 从特征名列表中删除已删除的列
        feature_names_updated = [fn for fn in feature_names if fn not in cols_to_drop_valid]
    else:
        # 如果未提供feature_names,使用训练集的列名
        feature_names_updated = X_train_filtered.columns.tolist()
    
    # ===================== 步骤6: 记录删除后的状态 =====================
    filtered_train_shape = X_train_filtered.shape
    filtered_test_shape = X_test_filtered.shape
    filtered_n_features = X_train_filtered.shape[1]
    
    n_features_dropped = original_n_features - filtered_n_features
    drop_rate = (n_features_dropped / original_n_features * 100) if original_n_features > 0 else 0
    
    if verbose:
        print(f"\n删除后:")
        print(f"  训练集形状: {filtered_train_shape}")
        print(f"  测试集形状: {filtered_test_shape}")
        print(f"  特征数量: {filtered_n_features}")
        print(f"\n删除统计:")
        print(f"  实际删除的特征数: {n_features_dropped}")
        print(f"  删除率: {drop_rate:.1f}%")
        print(f"  剩余特征数: {filtered_n_features}")
    
    # ===================== 步骤7: 数据完整性检查 =====================
    # 检查训练集和测试集的列是否一致
    train_cols = set(X_train_filtered.columns)
    test_cols = set(X_test_filtered.columns)
    
    if train_cols != test_cols:
        cols_only_train = train_cols - test_cols
        cols_only_test = test_cols - train_cols
        warning_msg = (
            f"\n⚠️ 警告: 删除后训练集和测试集的列不一致!\n"
            f"  仅在训练集: {len(cols_only_train)} 列\n"
            f"  仅在测试集: {len(cols_only_test)} 列"
        )
        if verbose:
            print(warning_msg)
    elif verbose:
        print(f"\n✓ 数据完整性检查通过: 训练集和测试集列完全一致")
    
    # ===================== 步骤8: 构建统计信息字典 =====================
    stats_dict = {
        'original_n_features': original_n_features,
        'filtered_n_features': filtered_n_features,
        'n_features_dropped': n_features_dropped,
        'drop_rate_percent': drop_rate,
        # 'cols_requested': len(columns_to_drop), # This might be misleading in keep mode if we consider requested to drop
        'cols_dropped_valid': len(cols_to_drop_valid),
        'cols_not_found': len(cols_not_in_train) + len(cols_not_in_test),
        'original_train_shape': original_train_shape,
        'filtered_train_shape': filtered_train_shape,
        'original_test_shape': original_test_shape,
        'filtered_test_shape': filtered_test_shape,
        'columns_dropped': cols_to_drop_valid,
        'data_consistent': train_cols == test_cols
    }
    
    if verbose:
        print(f"\n✅ 特征删除操作完成!")
        print("=" * 80)
    
    return X_train_filtered, X_test_filtered, feature_names_updated, stats_dict


In [ ]:
# 数据清洗脚本
from pipeline.perfect_pipeline import ConstantColumnDropper, ToxicityBinarizer
from pipeline.data_splitters import (
    PatientLevelStratifiedSplitter,
    PatientLevelStratifiedSplitterWithCV,
    patient_level_train_test_split
)

# 读入静态数据和动态数据
df = pd.read_csv("/home/phl/PHL/Car-T/model_v1/B-NHL_reindexed_example/csv/B-NHL_static_data.csv")
dynamic_data_dir = "/home/phl/PHL/Car-T/model_v1/B-NHL_reindexed_example/processed"
print(f"原始分布: {df.shape}")
# print (df)

# =============================================================================
#  1. 行处理: 剔除 Ann Arbor分期（Ann Arbor stage即AAS） 为空的患者（即白血病或者原发中枢的患者） 
# 原理：后续读取动态数据时（如 build_features_from_df 函数），是基于 df['ID'] 列表进行遍历的。因此只需要在内存中的静态数据DataFrame即 df 里剔除这些患者对应的行。
# 这样，后续的程序逻辑自然就会跳过这些 ID，从而达到“逻辑删除数据”的效果，而不会在物理上删除磁盘上的原始动态数据文件。
# =============================================================================
print("=" * 70)
# 找出 AAS为 NaN 的行
if 'AAS' in df.columns:
    missing_AAS_mask = df['AAS'].isna()
    missing_ids = df.loc[missing_AAS_mask, 'ID'].values
    
    if len(missing_ids) > 0:
        print(f"步骤1(行处理): 发现 {len(missing_ids)} 例患者 AAS 信息缺失(即白血病或原发中枢患者), 患者ID: {missing_ids}, 正在执行剔除操作...")
        
        # 仅从静态数据 DataFrame 中剔除这些行, 只要这里剔除了 ID，后续步骤自然会跳过读取对应的动态数据文件。
        df_clean1 = df[~missing_AAS_mask].reset_index(drop=True)
        
        print(f"已剔除变量 AAS 为空的患者")
        print(f"处理后分布: {df_clean1.shape}")
    else:
        print("检查通过：未发现 AAS 为空的患者。")
else:
    print("警告：静态数据中未找到 'AAS' 列，跳过此检查。")
# print (df_clean1)

# =============================================================================
#  2. 行处理: 剔除目标变量为空的患者(仅逻辑剔除静态变量中的数据，不删除物理文件)
# =============================================================================
print("=" * 70)   
# 步骤1: 处理目标变量列中的缺失值
if 'Infection' in df.columns:

    # 找出目标变量为空的患者ID并打印
    missing_label_mask = df_clean1['Infection'].isna()
    if missing_label_mask.sum() > 0:
        missing_ids_infection = df_clean1.loc[missing_label_mask, 'ID'].values
        print(f"步骤2(行处理): 发现 {df_clean1['Infection'].isna().sum()} 例患者目标变量列 'Infection' 缺失, 患者ID: {missing_ids_infection}, 正在执行剔除操作...")

    # 删除目标变量列中包含 NaN 的行，会导致索引ID不连续，但是不影响后续数据划分
    # 分割器返回的是索引位置(iloc），不依赖原始索引值
    df_clean2 = df_clean1[df_clean1['Infection'].notna()].copy()
    print(f"目标变量分布: {df_clean2['Infection'].value_counts().to_dict()}")
    print(f"处理后分布: {df_clean2.shape}")

    # 可选：重置索引使其连续（推荐用于后续处理），虽然索引不连续不影响划分，但如果后续代码依赖连续索引，可能会出问题
    df_clean2 = df_clean2.reset_index(drop=True)
# print(df_clean2)
# print(df_clean2.shape)

# ============================================================================
# 步骤3.1: 静态特征缺失值分析（先分析静态特征）
# ============================================================================
print("=" * 70)
print("步骤3.2(列处理): 静态特征缺失值分析")

df_clean2, static_missing_stats = filter_static_features_by_missing(
    df=df_clean2,
    missing_threshold=20.0,
    output_dir='/home/phl/PHL/Car-T/model_v1/output/static_feature_missing',
    verbose=True,
    drop_high_missing=False,  # 先分析（False），再决定是否删除（True）
    exclude_cols=['ID', 'Infection', 'CRS', 'ICANS', 'E_ICAHT', 'L_ICAHT']
)

print(f"\n步骤3.1完成: 静态特征分析完成")
print(f"\n原始形状: {static_missing_stats['original_shape']}")
print(f"\n静态特征数: {static_missing_stats['static_cols']}")
print(f"建议删除的静态特征列数: {static_missing_stats['cols_above_threshold']}")
print(f"实际删除的静态特征列数: {static_missing_stats['cols_dropped']}")

# ============================================================================
# 步骤3.2: 行处理: 执行动态特征提取并合并
# ============================================================================
print("=" * 70)
print("步骤3(行处理): 动态特征提取与合并")
# 1. 提取特征, df_clean2 是当前清洗好的静态数据框

# 增强版特征提取————创建监控器，指定要监控的变量
monitor = VariableMonitor(
    variables_to_monitor=['CBC001', 'CBC004'],  # 监控多个变量'baseline_CBC004'
    time_window=(-15, 0)                                               # 监控时间窗口
)

print(df_clean2)
baseline_features_df = extract_baseline_features(
    patient_ids=df_clean2['ID'].values,
    dynamic_dir=dynamic_data_dir,   # 使用动态数据定义路径
    time_col='day',                 # 请确认动态CSV中时间列名为 'day' 还是 'time'
    cutoff_day=0,                   # 截止时间点为0天（即治疗前）
    monitor=monitor                 # 传入监控器参数列表
)

# 获取所有变量的监控结果
all_empty_patients = monitor.get_empty_patient_ids()

print("\n所有变量的空数据统计:")
for var_name, patient_set in all_empty_patients.items():
    print(f"  {var_name}: {len(patient_set)} 个患者")
    if patient_set:
        print(f"    患者ID: {sorted(list(patient_set))[:5]}")  # 显示前5个
        
# 保存详细报告到文件
# monitor.print_summary()     # 打印摘要报告
monitor.save_report('/home/phl/PHL/Car-T/model_v1/output/monitor_report.json')


# 2. 合并到主数据框
# 使用左连接 (Left Join) 确保不会丢失 df_clean2 中的患者
# 如果某个患者没有动态数据，新列将自动填充为 NaN
print(f"\n合并前 df_clean2 形状: {df_clean2.shape}")
df_clean3 = pd.merge(df_clean2, baseline_features_df, on='ID', how='left')
print(f"合并后 df_clean3 形状: {df_clean3.shape}")

# 3. 简单的缺失值检查
new_cols = [c for c in df_clean3.columns if c.startswith('baseline_')]
missing_counts = df_clean3[new_cols].isna().sum()
print(f"\n新特征缺失值情况 (Top 5): \n{missing_counts.sort_values(ascending=False).head()}")
print(f"处理后分布: {df_clean3.shape}")
# print (df_clean3)

# ============================================================================
# 步骤3.3: 动态特征缺失值分析
# ============================================================================
df_clean3, dynamic_missing_stats = filter_baseline_features_by_missing(
    df=df_clean3,
    missing_threshold=20.0,
    output_dir='/home/phl/PHL/Car-T/model_v1/output/dynamic_feature_missing',
    verbose=True,
    drop_high_missing=False  # 先分析（False），再决定是否删除（True）
)

print(f"\n步骤3.3完成: 动态特征分析完成")
print(f"\n原始形状: {dynamic_missing_stats['original_shape']}")
print(f"\n动态特征数: {dynamic_missing_stats['baseline_cols']}")
print(f"建议删除的列数: {dynamic_missing_stats['cols_above_threshold']}")
print(f"实际删除的列数: {dynamic_missing_stats['cols_dropped']}")

# =============================================================================
# 4. 值处理: 二元化毒性等级
# =============================================================================
print("=" * 70)
print("步骤4(值处理): 二元化毒性等级")
# 删除常量列 & 缺失值处理
# dropper = ConstantColumnDropper()
# df_clean4 = dropper.fit_transform(df_clean3)
# 检查缺失值并删除包含缺失值的行
#df_cleaned = df_cleaned.dropna()

# binarizer = ToxicityBinarizer(col="CRS", threshold=2)   # 单个指标二元化
binarizer = ToxicityBinarizer(      # 多个毒性指标使用相同阈值
    columns=["CRS", "ICANS", "E_ICAHT", "L_ICAHT", "Infection"],
    threshold=2)
df_clean4 = binarizer.fit_transform(df_clean3)
# df_clean4 = binarizer.fit_transform(df_clean2)

# =============================================================================
#  5. 列处理: 手动删除不需要的冗余列（仅删除静态变量中的部分列）
# =============================================================================
print("=" * 70)
# 根据需要修改冗余列列表
cols_to_drop = ['CCID', 'Disease']

if cols_to_drop:
    print(f"步骤5(列处理): 正在删除冗余列: {cols_to_drop}")
    df_clean5 = df_clean4.drop(columns=cols_to_drop, errors='ignore') # errors='ignore' 防止列不存在时报错
    print(f"处理后分布: {df_clean5.shape}")
# print (df_clean5)
df_final = df_clean5

print(f"\n数据预览:{df_final.shape}")
print(f"行数:{df_final.shape[0]}，列数:{df_final.shape[1]}")
pd.set_option('display.max_columns', None)          # 显示所有列
pd.set_option('display.max_rows', None)             # 显示所有行
pd.set_option('display.expand_frame_repr', False)   #不换行显示
# 显示处理后的数据框，head() 方法本身就只返回前5行数据
# print(df_final.head())
# print(df_final)

In [ ]:
# 数据集划分
# 参考：https://sklearn.apachecn.org/master/30/————3.1.2.2.2. 分层随机 Split（StratifiedShuffleSplit）
# 方案1:70/30 患者级分层划分；
# 方案2:70/30 患者级分层划分 + 5折交叉验证

# =============================================================================
# 4. 数据集划分，划分为70%训练集和30%测试集，使用基于类的分割器进行患者级分层分割
# =============================================================================
print("=" * 70)
print("步骤6(数据集划分): 基础患者级分层划分")

# a.基础分割器
# splitter = PatientLevelStratifiedSplitter(test_size=0.3, random_state=42)
# train_df, test_df = splitter.split(
#     df_final, 
#     label_col="Infection", 
#     patient_id_col="ID"  # 确保患者级独立性
# )
# print(f"\n训练集大小: {len(train_df)}")
# print(f"测试集大小: {len(test_df)}")

# b.带交叉验证的分割器（5折交叉验证）
# #=====================================OLD版本输出差异（返回患者ID）=========================================
# # cv_folds 结构:
# cv_folds = [
#     (train_patient_ids, val_patient_ids),  # Fold 1
#     (train_patient_ids, val_patient_ids),  # Fold 2
#     ...]
# # 类型: List[Tuple[np.ndarray[str/int], np.ndarray[str/int]]]
# # 内容: 患者的ID值 (如 [101, 205, 308, ...])
# # 使用方式:
# for train_ids, val_ids in cv_folds:
#     # 必须手动转换: ID → DataFrame行
#     fold_train = train_df[train_df["patient_id"].isin(train_ids)]  # 慢!
#     fold_val = train_df[train_df["patient_id"].isin(val_ids)]
    
#     X_train = fold_train.drop(columns=["label"])
#     y_train = fold_train["label"]
# #=====================================NEW版本（返回整数索引）===========================================
# # 修改后: cv_folds_indices 直接是整数索引, 可直接用于 X_train.iloc[],且PatientLevelStratifiedSplitterWithCV 已重置了 train_df 索引，
#    所以X_train, y_train 实际上已经是 0,1,2...
# Sklearn标准用法: 返回整数索引而不是患者ID
# from sklearn.model_selection import StratifiedKFold
# skf = StratifiedKFold(n_splits=5)
# for train_idx, val_idx in skf.split(X, y):
#     X_train, X_val = X[train_idx], X[val_idx]  # 整数索引!
splitter = PatientLevelStratifiedSplitterWithCV(test_size=0.3, n_folds=3, random_state=42)    
train_df, test_df, cv_folds_indices = splitter.split(
    df_final, 
    label_col="Infection", 
    patient_id_col="ID"  # 确保患者级独立性
)
# 查看训练集和测试集分布
print("=" * 70)
print(f"数据划分完成:")
print(f"\n训练集大小: {len(train_df)} 样本")
print(f"\n训练集分布:\n{train_df}")
print(f"\n测试集大小: {len(test_df)} 样本")
print(f"\n测试集分布:\n{test_df}")
print(f"\n交叉验证折数: {len(cv_folds_indices)} 折")
print(f"\n交叉验证折数及索引:{cv_folds_indices}")

# 可选: 验证索引有效性 (首次运行时建议启用)
from pipeline.data_splitters import validate_cv_indices
validate_cv_indices(train_df, cv_folds_indices)

# # # ======================输出差异=====================
for train_idx, val_idx in cv_folds_indices:
    # 直接位置索引, 开箱即用!
    X_train = train_df.iloc[train_idx].drop(columns=["Infection"])
    y_train = train_df.iloc[train_idx]["Infection"]
    
    X_val = train_df.iloc[val_idx].drop(columns=["Infection"])
    y_val = train_df.iloc[val_idx]["Infection"]
    
    print(f"Fold 训练集大小: {len(X_train)}, 验证集大小: {len(X_val)}")
    print(f"训练集标签分布: {y_train.value_counts().to_dict()}")
    print(f"验证集标签分布: {y_val.value_counts().to_dict()}")
    print("-" * 50)
    print(f"\n特征数据预览:\n{X_train}")
    print(X_train.index.tolist())
    print(f"\n目标变量数据预览:\n{y_train}")
    print(y_train.index.tolist())

In [ ]:
# 将训练集和测试集保存为Excel文件

import os

# 定义保存路径
output_data_dir = '/home/phl/PHL/Car-T/model_v1/output/split_datasets'
if not os.path.exists(output_data_dir):
    os.makedirs(output_data_dir)
    print(f"创建输出目录: {output_data_dir}")

# # 保存训练集
# train_file_path = os.path.join(output_data_dir, 'train_dataset.xlsx')
# train_df.to_excel(train_file_path, index=False, engine='openpyxl')
# print(f"✅ 训练集已保存至: {train_file_path}")
# print(f"   形状: {train_df.shape}")

# # 保存测试集
# test_file_path = os.path.join(output_data_dir, 'test_dataset.xlsx')
# test_df.to_excel(test_file_path, index=False, engine='openpyxl')
# print(f"✅ 测试集已保存至: {test_file_path}")
# print(f"   形状: {test_df.shape}")

# 若数据集很大,Excel文件可能较大。可以考虑保存为CSV格式(更轻量)
train_df.to_csv(os.path.join(output_data_dir, 'train_dataset.csv'), index=False)
test_df.to_csv(os.path.join(output_data_dir, 'test_dataset.csv'), index=False)

# # 可选: 保存交叉验证折信息
# cv_info = []
# for i, (train_idx, val_idx) in enumerate(cv_folds_indices, 1):
#     cv_info.append({
#         'Fold': i,
#         'Train_Size': len(train_idx),
#         'Val_Size': len(val_idx),
#         'Train_Indices': str(train_idx.tolist()),
#         'Val_Indices': str(val_idx.tolist())
#     })

# cv_info_df = pd.DataFrame(cv_info)
# cv_file_path = os.path.join(output_data_dir, 'cv_folds_info.xlsx')
# cv_info_df.to_excel(cv_file_path, index=False, engine='openpyxl')
# print(f"✅ 交叉验证折信息已保存至: {cv_file_path}")

print(f"\n📁 所有文件已保存至目录: {output_data_dir}")


In [ ]:
# 五折交叉划分数据预处理————定义特征类型和清洗
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  # 导入缺失值填充器

from sklearn.base import BaseEstimator, TransformerMixin

# 自定义标签编码器，用于处理多列
# 在单元格 13 中替换 MultiColumnLabelEncoder

class MultiColumnLabelEncoder(BaseEstimator, TransformerMixin):
    """
    用于对多个类别列进行标签编码的自定义转换器
    支持 sklearn Pipeline 的 get_feature_names_out 方法
    """
    def __init__(self):
        self.label_encoders = {}
        self.feature_names_in_ = None
        
    def fit(self, X, y=None):
        """拟合每一列的标签编码器"""
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        self.feature_names_in_ = list(X_df.columns) if hasattr(X_df, 'columns') else [f"col_{i}" for i in range(X_df.shape[1])]
        
        for col_idx in range(X_df.shape[1]):
            le = LabelEncoder()
            le.fit(X_df.iloc[:, col_idx].astype(str))
            self.label_encoders[col_idx] = le
        return self
    
    def transform(self, X):
        """转换所有列"""
        X_df = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X
        X_transformed = np.zeros((X_df.shape[0], X_df.shape[1]), dtype=np.float64)
        
        for col_idx in range(X_df.shape[1]):
            le = self.label_encoders[col_idx]
            col_data = X_df.iloc[:, col_idx].astype(str)
            # 处理未见过的类别：映射为 -1
            X_transformed[:, col_idx] = col_data.map(
                lambda x: le.transform([x])[0] if x in le.classes_ else -1
            )
        
        return X_transformed
    
    def get_feature_names_out(self, input_features=None):
        """返回输出特征名称 - 兼容 sklearn Pipeline"""
        if input_features is not None:
            return np.array([f"cat__{name}" for name in input_features])
        elif self.feature_names_in_ is not None:
            return np.array([f"cat__{name}" for name in self.feature_names_in_])
        else:
            return np.array([f"cat__{i}" for i in range(len(self.label_encoders))])


# 分离特征和目标变量
X_train_raw = train_df.drop(columns=['Infection', 'ID',
                                    'CRS', 'ICANS','E_ICAHT', 'L_ICAHT' 
                                     ])
y_train = train_df['Infection']
y_train = train_df['Infection'].astype(int) # <--- 修改：强制转换为整数类型，XGBoost 因为数据类型（浮点标签）或配置原因没有正确广播自己是 classifier，会默认自己是 regressor，从而导致 AUC 0.5 的问题。

X_test_raw = test_df.drop(columns=['Infection', 'ID',
                                   'CRS', 'ICANS','E_ICAHT', 'L_ICAHT' 
                                    ])
y_test = test_df['Infection']
y_test = test_df['Infection'].astype(int) 

# =============================================================================
# 1. 精细化定义特征组
# =============================================================================

# A. 连续数值特征 -> StandardScaler
numeric_features = ['Age', 'BMDB']

# B. 无序分类特征（少类别）-> One-Hot 编码_OneHotEncoder——————字符串，无大小关系
nominal_features = ['CM', 'TYPE', 'PHSC']  # 类别数 ≤ 5

# # C. 无序分类特征（多类别）-> 标签编码_LabelEncoder（仅用于树模型）
# nominal_features_many = ['PHSC']  # 类别数 > 5

# D. 有序分类特征 -> OrdinalEncoder（保留顺序）
# 已经是 0,1,2 格式 -> 保持数值特性
# 这样 SVM/LR 可以利用其大小关系 (如 AAS 分期 4 > 1)，树模型也能更好切分
# 注意：请检查 PHSC, BMC 等是否真的是数值 0,1,2。如果是字符串，请移到 nominal_features
ordinal_features = ['Sex', 'BMC', 'EI', 'EM', 'B_symptoms', 'AAS', 'NL', 'BT', 'CTFA', 'PCT'
                    # 'CRS', 'ICANS','E_ICAHT', 'L_ICAHT'
                ]  # 有明确顺序的

# E. 二分类特征 -> 保持原值（无需编码）
# binary_features = ['Sex', 'BMC', 'EI', 'EM', 'B_symptoms', 'BT', 'CTFA', 'PCT']

# F. 自动识别动态特征 (baseline_*)，这些通常是数值型但包含脏数据

# 排除掉上面已经定义过的所有列
defined_features = numeric_features + nominal_features + ordinal_features
other_features = [col for col in X_train_raw.columns if col not in defined_features]
print(f"自动识别出的动态/其他特征: {len(other_features)} 个")

# =============================================================================
# 2. 数据清洗(需对连续数值特征、动态特征、类别特征分类型处理)
# =============================================================================
X_train = X_train_raw.copy()
X_test = X_test_raw.copy()

# 组1: 需要保持为数值的列 (连续数值 + 有序 + 动态)
cols_to_numeric = numeric_features + ordinal_features + other_features
print("正在清洗数值型和有序类别特征...")
# 将 DataFrame 中所有数据强制转数值，非数字(脏数据)变 NaN, 后续可以通过 SimpleImputer 统一填充这些缺失值
X_train[cols_to_numeric] = X_train[cols_to_numeric].apply(pd.to_numeric, errors='coerce')
X_test[cols_to_numeric] = X_test[cols_to_numeric].apply(pd.to_numeric, errors='coerce')

# B. 填充数值列的缺失值 (用中位数或0, 保持整数特性的列通常中位数也是整数)
# 计算训练集中位数，如果中位数也是NaN (说明整列都是NaN)，则填 0 (代表无数据)
train_medians = X_train[cols_to_numeric].median()
X_train[cols_to_numeric] = X_train[cols_to_numeric].fillna(train_medians).fillna(0)
X_test[cols_to_numeric] = X_test[cols_to_numeric].fillna(train_medians).fillna(0)

# 组2: 无序类别列 (字符串)(填充缺失值为 'missing'，保留字符串格式)
print("正在清洗无序类别特征...")
X_train[nominal_features] = X_train[nominal_features].fillna('missing').astype(str)
X_test[nominal_features] = X_test[nominal_features].fillna('missing').astype(str)

print(f"数据清洗完成。")
print(f"有序特征 (如 AAS) 示例值: {X_train['AAS'].unique()}")
print(f"无序特征 (如 TYPE) 示例值: {X_train['TYPE'].unique()}")
print(f"数据清洗完成。X_train 形状: {X_train.shape}")
print(f"数值列无缺失值: {not X_train[cols_to_numeric].isna().any().any()}")
print(f"类别列保留原始格式 (示例): {X_train['Sex'].unique()}")
# print(f"预处理后的训练数据：{X_train}")
# print(f"预处理后的测试数据：{X_test}")
print(f"预处理后 X_train 缺失值数量: {X_train.isna().sum().sum()}")
print(f"预处理后 X_test 缺失值数量: {X_test.isna().sum().sum()}")

# =============================================================================
# 3. 定义 Pipeline 预处理器
# https://zhuanlan.zhihu.com/p/42368821
# 构建一个包含预处理和模型的 Pipeline，扔给 GridSearchCV
# 不要在这里做 preprocessor.fit_transform，只做列类型的定义和基础清洗（行独立的操作）
# =============================================================================

# A.数值/有序特征预处理
# 归一化对 SVM/LR 很重要，对有序类别(0,1,2)做标准化也是安全的
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),      # 用中位数填充缺失值      
    ('scaler', StandardScaler())                        # 数值特征标准化
])

# ============================================================================
# B.无序类别特征预处理方案选择
# ============================================================================
# 方案1: 独热编码 (One-Hot Encoding) - 已注释，保留备用
# 优点: 不引入顺序假设，适合线性模型
# 缺点: 可能产生高维稀疏特征，增加特征间冗余和多重共线性
# ----------------------------------------------------------------------------
categorical_transformer = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # 用常数'missing'填充缺失值
    # drop='first' 适合线性模型(逻辑回归)，handle_unknown=PCgnore' 适合树模型
    # 如果主要用 XGBoost/RF，建议去掉 drop='first'，保留 handle_unknown='ignore'
    # sparse_output=False 适用于 sklearn >= 1.2，旧版本请用 sparse=False
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))         # 处理测试集中的新类别
])

# ----------------------------------------------------------------------------

# 方案2: 标签编码 (Label Encoding) - 当前使用
# 优点: 降低特征维度，减少冗余和多重共线性，提高计算效率
# 缺点: 引入了顺序假设（但树模型可以很好处理）
# 注意: 适合树模型（如XGBoost/RandomForest），对于线性模型建议使用独热编码
# ----------------------------------------------------------------------------
# 当前使用: 标签编码 (Label Encoding) 用于无序类别特征
# 适用于: 树模型 (XGBoost, RandomForest等)
# 若需使用独热编码，请取消注释上面的方案1代码并注释方案2
# categorical_transformer = Pipeline(steps=[
#     ('label_encoder', MultiColumnLabelEncoder())  # 标签编码
# ])


# # 少类别无序分类 -> OneHot
# nominal_few_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
# ])

# # 多类别无序分类 -> Label（仅树模型推荐）
# nominal_many_transformer = Pipeline(steps=[
#     ('label_encoder', MultiColumnLabelEncoder())
# ])

# # 有序分类 -> Ordinal
# ordinal_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
# ])

# # 二分类 -> 直接标准化（已经是 0/1）
# binary_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('scaler', StandardScaler())  # 可选：对二分类是否标准化取决于模型
# ])


# 组合预处理器 ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, cols_to_numeric),              # 将 连续数值 + 有序类别 + 动态特征 全部作为数值处理
        ('cat', categorical_transformer, nominal_features)      # 对 无序字符串 做 One-Hot 编码或标签编码
        # ('nom_few', nominal_few_transformer, nominal_features_few),
        # ('nom_many', nominal_many_transformer, nominal_features_many),
        # ('ord', ordinal_transformer, ordinal_features),
        # ('bin', binary_transformer, binary_features),
    ],
    # remainder='drop'            # 其他未定义的列丢弃
    remainder='passthrough'     # 重要：保留未在上面列出的其他列，否则会被丢弃；这里的 passthrough 会让清洗好的 baseline 特征直接通过
)

In [ ]:
# ==============================================================================
# 特征过滤步骤1:查看并保存 X_test、y_train、y_test 的信息
# ==============================================================================

import pandas as pd
import os

print("=" * 80)
print("X_test、y_train、y_test 信息查看与保存")
print("=" * 80)

output_dir = '/home/phl/PHL/Car-T/model_v1/output/feature_info'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# =============================================================================
# 1. X_test 列名保存
# =============================================================================
print("\n【1】X_test 列名保存")
print("-" * 80)

X_test_column_names = X_test.columns.tolist()

print(f"X_test 的形状: {X_test.shape}")
print(f"总列数: {len(X_test_column_names)}")

# 在控制台打印所有列名
print(f"\nX_test 所有列名:")
for i, col in enumerate(X_test_column_names, 1):
    print(f"  {i}. {col}")

# 按特征类型分组
baseline_cols_test = [col for col in X_test_column_names if col.startswith('baseline_')]
other_cols_test = [col for col in X_test_column_names if not col.startswith('baseline_')]

print(f"\nX_test 特征分布:")
print(f"  静态特征: {len(other_cols_test)} 个")
print(f"  动态特征 (baseline_*): {len(baseline_cols_test)} 个")

# 保存到txt文件
output_file_test = os.path.join(output_dir, 'X_test_column_names.txt')

with open(output_file_test, 'w', encoding='utf-8') as f:
    f.write("# X_test 列名清单\n")
    f.write(f"# 生成时间: {pd.Timestamp.now()}\n")
    f.write(f"# 数据形状: {X_test.shape}\n")
    f.write(f"# 总列数: {len(X_test_column_names)}\n")
    f.write("=" * 80 + "\n\n")
    
    f.write("所有列名列表:\n")
    f.write("-" * 80 + "\n")
    for i, col in enumerate(X_test_column_names, 1):
        f.write(f"{i}. {col}\n")
    
    f.write("\n" + "=" * 80 + "\n")
    f.write("特征类型分组:\n")
    f.write("-" * 80 + "\n")
    
    f.write(f"\n静态特征 ({len(other_cols_test)} 个):\n")
    for i, col in enumerate(other_cols_test, 1):
        f.write(f"  {i}. {col}\n")
    
    f.write(f"\n动态特征 ({len(baseline_cols_test)} 个):\n")
    for i, col in enumerate(baseline_cols_test, 1):
        f.write(f"  {i}. {col}\n")

print(f"\n✅ X_test 列名已保存至: {output_file_test}")

# 保存为CSV格式
csv_file_test = os.path.join(output_dir, 'X_test_column_names.csv')
pd.DataFrame({
    '序号': range(1, len(X_test_column_names) + 1),
    '列名': X_test_column_names,
    '特征类型': ['动态特征' if col.startswith('baseline_') else '静态特征' 
               for col in X_test_column_names]
}).to_csv(csv_file_test, index=False, encoding='utf-8-sig')

print(f"✅ X_test 列名CSV已保存至: {csv_file_test}")

# =============================================================================
# 2. y_train 信息保存
# =============================================================================
print("\n" + "=" * 80)
print("【2】y_train 信息保存")
print("-" * 80)

print(f"y_train 的形状: {y_train.shape}")
print(f"y_train 数据类型: {y_train.dtype}")
print(f"y_train 标签分布:\n{y_train.value_counts().to_dict()}")

# 保存到txt文件
output_file_ytrain = os.path.join(output_dir, 'y_train_info.txt')

with open(output_file_ytrain, 'w', encoding='utf-8') as f:
    f.write("# y_train 信息清单\n")
    f.write(f"# 生成时间: {pd.Timestamp.now()}\n")
    f.write(f"# 数据形状: {y_train.shape}\n")
    f.write(f"# 数据类型: {y_train.dtype}\n")
    f.write("=" * 80 + "\n\n")
    
    f.write("标签分布:\n")
    f.write("-" * 80 + "\n")
    for label, count in y_train.value_counts().sort_index().items():
        percentage = (count / len(y_train)) * 100
        f.write(f"  标签 {label}: {count} 个 ({percentage:.2f}%)\n")
    
    f.write(f"\n总样本数: {len(y_train)}\n")
    
    f.write("\n" + "=" * 80 + "\n")
    f.write("前20个样本预览:\n")
    f.write("-" * 80 + "\n")
    for i, val in enumerate(y_train.head(20), 1):
        f.write(f"  {i}. {val}\n")

print(f"\n✅ y_train 信息已保存至: {output_file_ytrain}")

# 保存为CSV格式
csv_file_ytrain = os.path.join(output_dir, 'y_train_data.csv')
pd.DataFrame({
    '索引': y_train.index,
    '标签': y_train.values
}).to_csv(csv_file_ytrain, index=False, encoding='utf-8-sig')

print(f"✅ y_train 数据CSV已保存至: {csv_file_ytrain}")

# =============================================================================
# 3. y_test 信息保存
# =============================================================================
print("\n" + "=" * 80)
print("【3】y_test 信息保存")
print("-" * 80)

print(f"y_test 的形状: {y_test.shape}")
print(f"y_test 数据类型: {y_test.dtype}")
print(f"y_test 标签分布:\n{y_test.value_counts().to_dict()}")

# 保存到txt文件
output_file_ytest = os.path.join(output_dir, 'y_test_info.txt')

with open(output_file_ytest, 'w', encoding='utf-8') as f:
    f.write("# y_test 信息清单\n")
    f.write(f"# 生成时间: {pd.Timestamp.now()}\n")
    f.write(f"# 数据形状: {y_test.shape}\n")
    f.write(f"# 数据类型: {y_test.dtype}\n")
    f.write("=" * 80 + "\n\n")
    
    f.write("标签分布:\n")
    f.write("-" * 80 + "\n")
    for label, count in y_test.value_counts().sort_index().items():
        percentage = (count / len(y_test)) * 100
        f.write(f"  标签 {label}: {count} 个 ({percentage:.2f}%)\n")
    
    f.write(f"\n总样本数: {len(y_test)}\n")
    
    f.write("\n" + "=" * 80 + "\n")
    f.write("前20个样本预览:\n")
    f.write("-" * 80 + "\n")
    for i, val in enumerate(y_test.head(20), 1):
        f.write(f"  {i}. {val}\n")

print(f"\n✅ y_test 信息已保存至: {output_file_ytest}")

# 保存为CSV格式
csv_file_ytest = os.path.join(output_dir, 'y_test_data.csv')
pd.DataFrame({
    '索引': y_test.index,
    '标签': y_test.values
}).to_csv(csv_file_ytest, index=False, encoding='utf-8-sig')

print(f"✅ y_test 数据CSV已保存至: {csv_file_ytest}")

# =============================================================================
# 4. 综合统计摘要
# =============================================================================
print("\n" + "=" * 80)
print("【4】综合统计摘要")
print("=" * 80)

summary_file = os.path.join(output_dir, 'data_summary.txt')

with open(summary_file, 'w', encoding='utf-8') as f:
    f.write("# 数据集综合统计摘要\n")
    f.write(f"# 生成时间: {pd.Timestamp.now()}\n")
    f.write("=" * 80 + "\n\n")
    
    f.write("1. X_train (训练特征集)\n")
    f.write("-" * 80 + "\n")
    f.write(f"   形状: {X_train.shape} (样本数 × 特征数)\n")
    f.write(f"   列数: {len(X_train.columns)}\n")
    f.write(f"   静态特征: {len([col for col in X_train.columns if not col.startswith('baseline_')])} 个\n")
    f.write(f"   动态特征: {len([col for col in X_train.columns if col.startswith('baseline_')])} 个\n\n")
    
    f.write("2. X_test (测试特征集)\n")
    f.write("-" * 80 + "\n")
    f.write(f"   形状: {X_test.shape} (样本数 × 特征数)\n")
    f.write(f"   列数: {len(X_test.columns)}\n")
    f.write(f"   静态特征: {len([col for col in X_test.columns if not col.startswith('baseline_')])} 个\n")
    f.write(f"   动态特征: {len([col for col in X_test.columns if col.startswith('baseline_')])} 个\n\n")
    
    f.write("3. y_train (训练标签)\n")
    f.write("-" * 80 + "\n")
    f.write(f"   形状: {y_train.shape}\n")
    f.write(f"   样本数: {len(y_train)}\n")
    f.write(f"   标签分布: {y_train.value_counts().to_dict()}\n\n")
    
    f.write("4. y_test (测试标签)\n")
    f.write("-" * 80 + "\n")
    f.write(f"   形状: {y_test.shape}\n")
    f.write(f"   样本数: {len(y_test)}\n")
    f.write(f"   标签分布: {y_test.value_counts().to_dict()}\n\n")
    
    f.write("5. 数据集划分比例\n")
    f.write("-" * 80 + "\n")
    total_samples = len(X_train) + len(X_test)
    train_ratio = (len(X_train) / total_samples) * 100
    test_ratio = (len(X_test) / total_samples) * 100
    f.write(f"   总样本数: {total_samples}\n")
    f.write(f"   训练集: {len(X_train)} ({train_ratio:.2f}%)\n")
    f.write(f"   测试集: {len(X_test)} ({test_ratio:.2f}%)\n")

print(f"\n✅ 综合统计摘要已保存至: {summary_file}")

# =============================================================================
# 总结
# =============================================================================
print("\n" + "=" * 80)
print("✅ 所有数据信息保存完成!")
print("=" * 80)
print(f"\n保存的文件列表:")
print(f"  1. X_train_column_names.txt")
print(f"  2. X_train_column_names.csv")
print(f"  3. X_test_column_names.txt")
print(f"  4. X_test_column_names.csv")
print(f"  5. y_train_info.txt")
print(f"  6. y_train_data.csv")
print(f"  7. y_test_info.txt")
print(f"  8. y_test_data.csv")
print(f"  9. data_summary.txt (综合摘要)")
print(f"\n保存路径: {output_dir}")
print("=" * 80)

In [ ]:
# =============================================================================
# # 特征过滤步骤2:基于特征选择结果的数据集过滤模块
# =============================================================================
# 作用: 根据特征选择分析结果,从原始数据中筛选出10个关键特征
# 输入: X_train, X_test (清洗后的原始DataFrame), selected_features (特征列表)
# 输出: X_train_filtered, X_test_filtered (仅包含选定特征的DataFrame)
# =============================================================================

import pandas as pd
import numpy as np
import os

print("=" * 80)
print("基于特征选择结果的数据集过滤")
print("=" * 80)

# =============================================================================
# 步骤 1: 定义/加载选定的10个特征
# =============================================================================
print("\n步骤 1: 确定要保留的 10 个特征")

# 方案1: 手动指定特征名称列表（基于您的特征选择结果）
# 注意: 这里使用的是**原始列名**（清洗后、预处理前的列名）
selected_features = [
    'CTFA',                         # 静态特征: 自体移植序贯
    'EI',                           # 静态特征: 结外有无病变
    'CM',                           # 静态特征: CAR-T共刺激分子
    'PCT',                          # 静态特征: 既往有无CAR-T治疗
    'TYPE',                         # 静态特征: CAR-T结构类型
    'baseline_CBC013',              # 动态特征: 血红蛋白
    'baseline_Vital Signs001',      # 动态特征: 体温
    'baseline_Electrolytes004',     # 动态特征: 校正钙
    'baseline_Biochemistry002',     # 动态特征: 天门冬氨酸氨基转移酶
    'baseline_Coagulation006',      # 动态特征: 凝血酶时间
]

# 方案2: 从特征选择结果文件中自动加载（推荐）
# feature_selection_result_path = '/home/phl/PHL/Car-T/model_v1/output/feature_selection/selected_features.txt'
# try:
#     with open(feature_selection_result_path, 'r', encoding='utf-8') as f:
#         selected_features = [line.strip() for line in f if line.strip() and not line.startswith('#')]
#     print(f"  ✅ 从文件加载了 {len(selected_features)} 个特征")
# except Exception as e:
#     print(f"  ⚠️ 自动加载失败: {e}, 使用手动指定的特征列表")

print(f"\n选定的 {len(selected_features)} 个特征:")
for i, feat in enumerate(selected_features, 1):
    feat_type = "动态特征" if feat.startswith('baseline_') else "静态特征"
    print(f"  {i}. {feat:30s} ({feat_type})")

# =============================================================================
# 步骤 2: 验证特征是否存在于原始数据集中
# =============================================================================
print("\n步骤 2: 验证特征有效性...")

# 检查训练集
missing_in_train = [f for f in selected_features if f not in X_train.columns]
if missing_in_train:
    print(f"  ⚠️ 警告: 以下特征在训练集中不存在:")
    for feat in missing_in_train:
        print(f"    - {feat}")
    print(f"\n  尝试从可用特征中寻找相似特征...")
    print(f"  训练集可用特征示例: {list(X_train.columns[:10])}")

# 检查测试集
missing_in_test = [f for f in selected_features if f not in X_test.columns]
if missing_in_test:
    print(f"  ⚠️ 警告: 以下特征在测试集中不存在:")
    for feat in missing_in_test:
        print(f"    - {feat}")

# 移除不存在的特征（确保训练集和测试集一致性）
valid_features = [f for f in selected_features if f in X_train.columns and f in X_test.columns]

if len(valid_features) < len(selected_features):
    print(f"\n  已自动移除不存在的特征")
    print(f"  原始特征数: {len(selected_features)}")
    print(f"  有效特征数: {len(valid_features)}")
    
    # 如果有效特征不足10个，从剩余特征中补充
    if len(valid_features) < 10:
        print(f"\n  ⚠️ 有效特征不足10个，从剩余特征中补充...")
        
        # 排除已选和不应包含的列
        exclude_cols = ['ID', 'Infection', 'CRS', 'ICANS', 'E_ICAHT', 'L_ICAHT']
        available_features = [f for f in X_train.columns 
                            if f not in valid_features 
                            and f not in exclude_cols
                            and f in X_test.columns]  # 确保测试集也有
        
        n_needed = 10 - len(valid_features)
        additional_features = available_features[:n_needed]
        valid_features.extend(additional_features)
        
        print(f"  补充的特征: {additional_features}")
else:
    valid_features = selected_features
    print(f"  ✅ 所有选定特征均存在于训练集和测试集中")

# =============================================================================
# 步骤 3: 从原始数据中筛选特征（关键步骤）
# =============================================================================
print(f"\n步骤 3: 从原始数据中筛选特征...")

# 记录原始形状
original_train_shape = X_train.shape
original_test_shape = X_test.shape

# 【关键】直接覆盖全局变量（仅保留选定的列）
# 注意: 这是在预处理（Pipeline）之前进行的过滤
X_train = X_train[valid_features].copy()
X_test = X_test[valid_features].copy()

print(f"\n  特征过滤结果:")
print(f"    训练集: {original_train_shape} → {X_train.shape}")
print(f"    测试集: {original_test_shape} → {X_test.shape}")
print(f"    特征减少: {original_train_shape[1]} → {X_train.shape[1]} " +
      f"(减少 {original_train_shape[1] - X_train.shape[1]} 个特征, " +
      f"保留 {X_train.shape[1] / original_train_shape[1] * 100:.1f}%)")

# =============================================================================
# 步骤 4: 数据质量检查
# =============================================================================
print(f"\n步骤 4: 数据质量检查...")

# 4.1 检查缺失值
train_missing = X_train.isna().sum().sum()
test_missing = X_test.isna().sum().sum()

print(f"\n  【缺失值统计】")
print(f"    训练集: {train_missing} 个 ({train_missing / X_train.size * 100:.2f}%)")
print(f"    测试集: {test_missing} 个 ({test_missing / X_test.size * 100:.2f}%)")

if train_missing > 0 or test_missing > 0:
    print(f"\n  各特征的缺失情况:")
    missing_stats = pd.DataFrame({
        '特征名': valid_features,
        '训练集缺失': [X_train[f].isna().sum() for f in valid_features],
        '训练集缺失率(%)': [X_train[f].isna().sum() / len(X_train) * 100 for f in valid_features],
        '测试集缺失': [X_test[f].isna().sum() for f in valid_features],
        '测试集缺失率(%)': [X_test[f].isna().sum() / len(X_test) * 100 for f in valid_features]
    })
    missing_stats = missing_stats[(missing_stats['训练集缺失'] > 0) | (missing_stats['测试集缺失'] > 0)]
    if not missing_stats.empty:
        print(missing_stats.to_string(index=False))
    print(f"\n  💡 提示: 缺失值将由Pipeline中的SimpleImputer自动处理")

# 4.2 数据类型检查
print(f"\n  【数据类型分布】")
print(f"    训练集:")
print(X_train.dtypes.value_counts().to_string())

# 4.3 检查训练集和测试集特征一致性
assert list(X_train.columns) == list(X_test.columns), \
    "❌ 错误: 训练集和测试集的特征列不一致!"
print(f"\n  ✅ 训练集和测试集特征列完全一致")

# 4.4 基本统计信息
print(f"\n  【训练集统计摘要】")
print(X_train.describe().T.to_string())

# =============================================================================
# 步骤 5: 更新 preprocessor 的特征列表（自动适配）
# =============================================================================
print(f"\n步骤 5: 更新预处理器配置...")

# 重新定义特征组（基于筛选后的列）
numeric_features_new = [f for f in ['Age', 'BMDB'] if f in X_train.columns]
nominal_features_new = [f for f in ['CM', 'TYPE', 'PHSC'] if f in X_train.columns]
ordinal_features_new = [f for f in ['AAS', 'Sex', 'BMC', 'EI', 'NL', 'EM', 
                                     'B_symptoms', 'BT', 'CTFA', 'PCT'] 
                        if f in X_train.columns]

# 动态特征（baseline_*）
other_features_new = [col for col in X_train.columns 
                      if col not in numeric_features_new + nominal_features_new + ordinal_features_new]

print(f"\n  更新后的特征分组:")
print(f"    数值特征 ({len(numeric_features_new)}): {numeric_features_new}")
print(f"    无序类别 ({len(nominal_features_new)}): {nominal_features_new}")
print(f"    有序类别 ({len(ordinal_features_new)}): {ordinal_features_new}")
print(f"    动态特征 ({len(other_features_new)}): {other_features_new[:5]}..." if len(other_features_new) > 5 else f"    动态特征 ({len(other_features_new)}): {other_features_new}")

# 重新构建 ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

cols_to_numeric_new = numeric_features_new + ordinal_features_new + other_features_new

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, cols_to_numeric_new),
        ('cat', categorical_transformer, nominal_features_new)
    ],
    remainder='passthrough'
)

print(f"\n  ✅ preprocessor 已更新为适配 {X_train.shape[1]} 个特征的配置")

# =============================================================================
# 步骤 6: 保存过滤后的数据集和元数据
# =============================================================================
print(f"\n步骤 6: 保存过滤后的数据集...")

output_dir_filtered = '/home/phl/PHL/Car-T/model_v1/output/filtered_features_data'
if not os.path.exists(output_dir_filtered):
    os.makedirs(output_dir_filtered)

# 6.1 保存特征列表
with open(os.path.join(output_dir_filtered, 'filtered_features_list.txt'), 'w', encoding='utf-8') as f:
    f.write("# 模型训练使用的过滤后特征列表\n")
    f.write(f"# 生成时间: {pd.Timestamp.now()}\n")
    f.write(f"# 原始特征数: {original_train_shape[1]}\n")
    f.write(f"# 过滤后特征数: {len(valid_features)}\n")
    f.write(f"# 特征减少率: {(original_train_shape[1] - len(valid_features)) / original_train_shape[1] * 100:.1f}%\n")
    f.write("=" * 80 + "\n\n")
    
    for i, feat in enumerate(valid_features, 1):
        feat_type = "动态特征" if feat.startswith('baseline_') else "静态特征"
        f.write(f"{i}. {feat:30s} ({feat_type})\n")

# 6.2 保存数据集
X_train.to_csv(os.path.join(output_dir_filtered, 'X_train_filtered.csv'), index=False)
X_test.to_csv(os.path.join(output_dir_filtered, 'X_test_filtered.csv'), index=False)

# 6.3 保存标签
pd.Series(y_train, name='Infection').to_csv(
    os.path.join(output_dir_filtered, 'y_train.csv'), index=False
)
pd.Series(y_test, name='Infection').to_csv(
    os.path.join(output_dir_filtered, 'y_test.csv'), index=False
)

# 6.4 保存特征类型映射
feature_type_mapping = pd.DataFrame({
    '特征名': valid_features,
    '特征类型': ['动态特征' if f.startswith('baseline_') else '静态特征' for f in valid_features],
    '数据类型': [str(X_train[f].dtype) for f in valid_features],
    '训练集缺失数': [X_train[f].isna().sum() for f in valid_features],
    '测试集缺失数': [X_test[f].isna().sum() for f in valid_features]
})
feature_type_mapping.to_csv(
    os.path.join(output_dir_filtered, 'feature_type_mapping.csv'), 
    index=False, encoding='utf-8-sig'
)

print(f"\n  ✅ 数据已保存至: {output_dir_filtered}")
print(f"    - X_train_filtered.csv ({X_train.shape})")
print(f"    - X_test_filtered.csv ({X_test.shape})")
print(f"    - y_train.csv ({len(y_train)} 样本)")
print(f"    - y_test.csv ({len(y_test)} 样本)")
print(f"    - filtered_features_list.txt")
print(f"    - feature_type_mapping.csv")

# =============================================================================
# 步骤 7: 防止数据泄露的验证检查
# =============================================================================
print(f"\n步骤 7: 数据泄露风险检查...")

# 7.1 确认没有使用测试集信息进行特征选择
print(f"\n  【数据泄露检查清单】")
print(f"    ✓ 特征选择基于: 训练集 (X_train)")
print(f"    ✓ 特征过滤顺序: 先划分数据集 → 再进行特征选择")
print(f"    ✓ 标准化/填充: 仅在训练集上fit，测试集上transform")
print(f"    ✓ 特征一致性: 训练集和测试集使用相同的特征列表")

# 7.2 检查是否有目标变量泄露
target_related_features = [f for f in valid_features if any(
    keyword in f.lower() for keyword in ['infection', 'crs', 'icans', 'icaht', 'outcome', 'result']
)]
if target_related_features:
    print(f"\n  ⚠️ 警告: 发现可能与目标变量相关的特征:")
    for feat in target_related_features:
        print(f"    - {feat}")
    print(f"  建议: 检查这些特征是否在预测时可用")
else:
    print(f"\n  ✅ 未发现明显的目标变量泄露特征")

# =============================================================================
# 总结
# =============================================================================
print("\n" + "=" * 80)
print("✅ 特征过滤完成!")
print("=" * 80)

print(f"\n【关键信息摘要】")
print(f"  原始特征数: {original_train_shape[1]}")
print(f"  过滤后特征数: {len(valid_features)}")
print(f"  特征保留率: {len(valid_features) / original_train_shape[1] * 100:.1f}%")
print(f"  静态特征: {len([f for f in valid_features if not f.startswith('baseline_')])} 个")
print(f"  动态特征: {len([f for f in valid_features if f.startswith('baseline_')])} 个")

print(f"\n【数据集信息】")
print(f"  训练集: {X_train.shape} (样本数 × 特征数)")
print(f"  测试集: {X_test.shape} (样本数 × 特征数)")
print(f"  标签分布: {dict(zip(*np.unique(y_train, return_counts=True)))}")

print(f"\n【后续步骤】")
print(f"  1. X_train 和 X_test 已更新为仅包含 {len(valid_features)} 个特征")
print(f"  2. preprocessor 已自动重新配置以适配新特征")
print(f"  3. 后续的模型训练代码将自动使用过滤后的数据")
print(f"  4. Pipeline 结构: preprocessor → final_imputer → classifier")
print(f"  5. 所有中间结果已保存至: {output_dir_filtered}")

print("=" * 80)

## 超参数优化
https://cloud.baidu.com/article/5601757

### 一、超参数优化：机器学习模型性能提升的核心引擎
在机器学习模型开发过程中，超参数优化是连接算法理论与实际性能的关键桥梁。不同于通过数据训练自动调整的模型参数（如神经网络权重），超参数需在训练前预先设定，直接影响模型的学习能力与泛化性能。例如，决策树的深度、支持向量机的核函数类型、神经网络的学习率与层数等，均属于需要人工调优的超参数范畴。

超参数优化的本质是通过系统化探索超参数空间，寻找使模型在验证集或测试集上表现最优的参数组合。这一过程不仅能显著提升模型精度，还可避免因参数设置不当导致的过拟合或欠拟合问题。以图像分类任务为例，通过优化卷积神经网络的超参数（如滤波器数量、步长、正则化系数），模型在CIFAR-10数据集上的准确率可从75%提升至89%。

### 二、网格搜索：系统化穷举的经典方法
1. 原理与实现逻辑
网格搜索（Grid Search）通过定义超参数的候选值集合，构建所有可能的参数组合，并逐一评估模型性能。例如，若需优化学习率（取值为{0.01, 0.001, 0.0001}）和正则化系数（取值为{0.1, 0.01, 0.001}），则需训练3×3=9个模型，最终选择验证集上损失最小的组合。

2. 优缺点分析
- 优势：
    - 覆盖全面，避免遗漏潜在最优解
    - 实现简单，适用于参数空间较小（<10个参数）或离散值较多的场景
- 局限：
    - 计算成本随参数数量呈指数增长（如10个参数，每个取3个值，需评估3^10=59,049次）
    - 对连续参数需预先离散化，可能错过最优区间


In [ ]:
# 完整的交叉验证训练逻辑：集成了参数搜索、全量重训和结果保存————改进版
# ✅ 移除 ID 转换逻辑: 新版本直接返回整数索引，无需转换
# ✅ 使用 .iloc[indices]: 直接使用位置索引
# ✅ LightGBM 改用 LGBMClassifier: 确保 sklearn 接口兼容性

# 为了实现预期工作流（CV找参数 -> 提取最佳参数 -> 全量重训 -> 保存），最标准且稳健的方法是使用 sklearn.model_selection.GridSearchCV（网格搜索）。GridSearchCV 的 refit=True 参数完美解决了这个问题：
# 1.自动执行交叉验证寻找最佳参数。
# 2.找到最佳参数后，自动使用这些参数在完整的 X_train 上重新训练模型。
# 3.grid.best_estimator_ 就是最终模型。

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# =============================================================================
# 1. 定义模型及其超参数搜索空间
# =============================================================================
# 注意：为了演示速度，这里参数范围设置较小。您可以根据需要扩展列表。
model_params = {
    # 'Tabnet': {
    #     'model': Pipeline([
    #         ('preprocessor', preprocessor),
    #         ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
    #         ('classifier', TabNetClassifier(
    #             seed=42,
    #             verbose=0  # 静默模式
    #         ))
    #     ]),
    #     'params': {
    #         'classifier__lr': [0.02, 0.05],
    #         'classifier__max_epochs': [50, 100],
    #         'classifier__batch_size': [256, 512]
    #     }
    # },
    'CatBoost': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', CatBoostClassifier(
                random_seed=42,
                verbose=0  # 静默模式
            ))
        ]),
        'params': {
            'classifier__learning_rate': [0.05, 0.1],
            'classifier__depth': [4, 6],
            'classifier__iterations': [100, 200]
        }
    },    
    # 'GLM': {
    #     'model': Pipeline([
    #         ('preprocessor', preprocessor),
    #         ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
    #         ('classifier', sm.GLM(random_state=42, max_iter=1000))
    #     ]),
    #     'params': {
    #         'classifier__C': [0.1, 1, 10],
    #         'classifier__penalty': ['l2', 'none'],
    #         'classifier__solver': ['lbfgs', 'saga']
    #     }
    # },
    'SVM': {        
        # 原始步骤
        # 'model': SVC(probability=True, random_state=42),
        # 'params': {
        #     'C': [0.1, 1, 10],
        #     'kernel': ['rbf', 'linear']
        # }
        # 改进版步骤：将预处理和模型打包
        'model': Pipeline([
            ('preprocessor', preprocessor),
            # 针对 passthrough 列的额外缺失值处理（防止漏网之鱼）
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', SVC(probability=True, random_state=42))
        ]),
        'params': {
            # 注意参数前缀要加上步骤名称 (classifier__)
            'classifier__C': [0.1, 1, 10],
            'classifier__kernel': ['rbf', 'linear']
        }
    },
    
    # XGBoost 的得分为 0.500000 且标准差为 0.000000，这说明模型完全没有学到任何东西，或者预测结果全是同一个类别（例如全预测为0）。
    # 这通常是因为列名包含特殊字符（如 [、]、< 等），XGBoost对特征名非常敏感，而其他模型（如 SVM、RF）则不敏感。当特征名不合法时，XGBoost可能会静默失败或表现异常。
    'XGBoost': {
        'model': Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
        # 修改点：移除了 use_label_encoder=False 和 eval_metric
        # 新版 XGBoost 默认就是 binary:logistic，且不需要手动指定 label_encoder
        # ('classifier', xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False, random_state=42))
        ('classifier', xgb.XGBClassifier(
            objective='binary:logistic', 
            random_state=42,
            n_jobs=1  # 关键：设为1,强制XGBoost使用单线程，因为外层 GridSearchCV 已经是并行了
            ))
        ]),
        'params': {
            'classifier__learning_rate': [0.05, 0.1],
            'classifier__max_depth': [3, 5],
            'classifier__n_estimators': [100, 200]
        }
    },
    
    # boosting_type='gbdt',
    # max_depth=-1,
    # min_data_in_leaf=20,
    'LightGBM': {
        'model': Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
        ('classifier', lgb.LGBMClassifier(
            objective='binary', 
            metric='binary_logloss', 
            verbose=-1, 
            random_state=42, 
            n_jobs=1  # 关键：设为1,强制LightGBM使用单线程，因为外层 GridSearchCV 已经是并行了
            ))
        ]),
        'params': {
            'classifier__learning_rate': [0.05, 0.1],
            'classifier__n_estimators': [100, 200],
            'classifier__num_leaves': [31, 50]
        }
    },
    'Logistic Regression': {
        'model': Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
        ('classifier', LogisticRegression(random_state=42, max_iter=1000))
        ]),
        'params': {
            'classifier__C': [0.1, 1, 10],
            'classifier__solver': ['liblinear', 'lbfgs']
        }
    },
    'Random Forest': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('classifier', RandomForestClassifier(random_state=42))
        ]),
        'params': {
            'classifier__n_estimators': [100, 200],
            'classifier__max_depth': [None, 10, 20]
        }
    },
    'MLP': {
        'model': Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', MLPClassifier(max_iter=500, random_state=42))
        ]),
        'params': {
            'classifier__hidden_layer_sizes': [(100,), (50, 50)],
            'classifier__alpha': [0.0001, 0.001]
        }
    },
    'KNN': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', KNeighborsClassifier())
        ]),
        'params': {
            # 'classifier__n_neighbors': [3, 5, 7],     # 视训练集的样本数量而定
            'classifier__n_neighbors': [1, 3],
            'classifier__weights': ['uniform', 'distance']
        }
    },
    'AdaBoost': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', AdaBoostClassifier(algorithm='SAMME', random_state=42))
        ]),
        'params': {
            'classifier__n_estimators': [50, 100],
            'classifier__learning_rate': [0.1, 0.2, 0.5, 1.0]
        }
    },
    'Decision Tree': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', DecisionTreeClassifier(random_state=42))
        ]),
        'params': {
            'classifier__max_depth': [None, 10, 20],
            'classifier__min_samples_split': [2, 5]
        }
    },
    'Naive Bayes': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', GaussianNB())
        ]),
        'params': {
            'classifier__var_smoothing': [1e-9, 1e-8]
        }
    },
    'Gradient Boosting': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', GradientBoostingClassifier(random_state=42))
        ]),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__learning_rate': [0.05, 0.1],
            'classifier__max_depth': [3, 5, 7]
        }
    },

}


# =============================================================================
# 2. 执行 GridSearchCV (CV + 自动重训)
# =============================================================================
# 使用第1折进行训练
# # ========== OLD版本代码 ==========
# train_df, test_df, cv_folds = splitter.split(df, "label", "patient_id")
# train_patient_ids, val_patient_ids = cv_folds[0]                          # 第1折
# fold1_train = train_df[train_df["patient_id"].isin(train_patient_ids)]    # 需要手动筛选
# fold1_val = train_df[train_df["patient_id"].isin(val_patient_ids)]

# X_train = fold1_train.drop(columns=["label", "patient_id"])
# y_train = fold1_train["label"]
# # ========== NEW版本代码==========
# # train_df, test_df, cv_folds = splitter.split(df, "label", "ID")
# train_idx, val_idx = cv_folds_indices[0]  # 第1折 - 直接索引!

# X_train = train_df.iloc[train_idx].drop(columns=["Infection", "ID"])
# y_train = train_df.iloc[train_idx]["Infection"]


# 模型训练
# # ========== OLD版本代码 ==========
# train_df, test_df, cv_folds = splitter.split(df, "label", "patient_id")

# for fold_idx, (train_ids, val_ids) in enumerate(cv_folds):
#     fold_train = train_df[train_df["patient_id"].isin(train_ids)] # 手动转换ID → 行
#     fold_val = train_df[train_df["patient_id"].isin(val_ids)]
    
#     # 训练模型
#     model.fit(fold_train[features], fold_train["label"])

# ========== NEW版本代码 (2处修改) ==========
# train_df, test_df, cv_folds = splitter.split(df, "Infection", "ID")  # 修改1: 列名

# for fold_idx, (train_idx, val_idx) in enumerate(cv_folds_indices):  # 修改2: 变量名
#     # 直接索引 (替换 isin 为 iloc)
#     fold_train = train_df.iloc[train_idx]
#     fold_val = train_df.iloc[val_idx]
    
#     # 训练模型 (无需修改)
#     svm = SVC(probability=True, random_state=42)
#     # svm.fit(X_train,y_train)
#     svm.fit(fold_train[features], fold_train["Infection"])

# 使用预处理后的 X_train 和 y_train
print(f"✅ 开始超参数调优与最终模型训练...")
print(f"使用数据: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"交叉验证策略: 使用预定义的 cv_folds_indices ({len(cv_folds_indices)} 折)")

# 存储优化后的结果
final_models = {}      # 存储重训后的最佳模型
cv_results_data = []   # 存储绘图数据

# 遍历每个模型进行网格搜索
for model_name, config in model_params.items():
    print(f"\n正在优化 {model_name} ...")
    
    # 核心：GridSearchCV
    # cv=cv_folds_indices: 确保使用和之前完全一致的切分
    # refit=True (默认): 找到最佳参数后，自动在整个 X_train 上重训
    gs = GridSearchCV(
        estimator=config['model'],
        param_grid=config['params'],
        cv=cv_folds_indices,
        scoring='roc_auc',
        n_jobs=-1,  # 并行计算，可加速训练（使用所有 CPU 核心）
        refit=True,
        verbose=1,
        error_score='raise' # 如果出错，抛出异常而不是静默失败
    )
    
    gs.fit(X_train, y_train)
    
    # 1. 保存最佳模型 (已在全量数据上训练好)
    final_models[model_name] = gs.best_estimator_
    
    # 2. 记录结果
    best_score = gs.best_score_
    best_params = gs.best_params_
    best_std = gs.cv_results_['std_test_score'][gs.best_index_]
    
    print(f"  -> 最佳 AUC: {best_score:.4f} (±{best_std:.4f})")
    print(f"  -> 最佳参数: {best_params}")
    
    # 收集每一折的详细分数用于绘图 (提取最佳参数对应的那些折的分数)
    # split0_test_score, split1_test_score ...
    fold_scores = []
    for i in range(len(cv_folds_indices)):
        fold_scores.append(gs.cv_results_[f'split{i}_test_score'][gs.best_index_])
        
    cv_results_data.append({
        'Model': model_name,
        'Mean AUC': best_score,
        'Std AUC': best_std,
        'Scores': fold_scores,
        'Best Params': str(best_params)
    })
    
    print(f"Best AUC for {model_name}: {best_score:.4f} (+/- {best_std:.4f})")
    print(f"Best Params: {best_params}\n")
# =============================================================================
# 3. 结果汇总与保存
# =============================================================================
print("\n" + "="*80)
print("最终优化结果汇总")
print("="*80)

# 将结果转换为 DataFrame
df_results = pd.DataFrame(cv_results_data).sort_values('Mean AUC', ascending=False)
print(df_results[['Model', 'Mean AUC', 'Std AUC', 'Best Params']].to_string(index=False))

# 保存结果到 CSV
output_dir = '/home/phl/PHL/Car-T/model_v1/output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
df_results.drop(columns=['Scores']).to_csv(os.path.join(output_dir, 'cv_optimization_results.csv'), index=False)

# =============================================================================
# 4. 可视化
# =============================================================================
fig, ax = plt.subplots(figsize=(12, 6))
models_list = df_results['Model'].tolist()
means = df_results['Mean AUC'].tolist()
stds = df_results['Std AUC'].tolist()
x_pos = np.arange(len(models_list))

ax.bar(x_pos, means, yerr=stds, align='center', alpha=0.7, 
       capsize=10, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b'])
ax.set_ylabel('AUC Score (CV Optimized)', fontsize=12)
ax.set_title('Best Cross-Validation Performance after Tuning', fontsize=14)
ax.set_xticks(x_pos)
ax.set_xticklabels(models_list, rotation=45, ha='right')
ax.set_ylim([0.5, 1.0])
ax.yaxis.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'cv_optimized_barplot.png'), dpi=300)
plt.show()

# =============================================================================
# 5. 将最佳模型赋值给全局变量
# =============================================================================
# 这样后续单元格直接使用 svm, rf 等变量时，用的就是调优后的最佳模型
svm = final_models['SVM']
xgb_model = final_models['XGBoost']
lgb_model = final_models['LightGBM']
log_reg = final_models['Logistic Regression']
rf = final_models['Random Forest']
mlp = final_models['MLP']
knn = final_models['KNN']
ada = final_models['AdaBoost']
dt = final_models['Decision Tree']
nb = final_models['Naive Bayes']

print("\n所有模型已使用最佳参数在完整训练集上重新训练完成！")
print("全局变量 (svm, rf, xgb_model...) 已更新为最佳模型。")

### 四、贝叶斯优化：智能导向的先进方法
1. 原理与核心逻辑
贝叶斯优化（Bayesian Optimization）通过构建目标函数（如模型准确率）的概率代理模型（通常为高斯过程），结合采集函数（如Expected Improvement）动态选择下一个评估点。其核心优势在于“记忆性”：每次评估后更新代理模型，引导后续搜索向高潜力区域集中。

2. 优缺点分析
- 优势：
    - 计算效率极高，尤其适用于评估成本高昂的场景（如深度学习模型）
    - 自动平衡探索（新区域）与利用（已知高潜力区域）
- 局限：
    - 代理模型构建复杂，需调整超参数（如高斯过程的核函数）
    - 初始阶段可能因代理模型不准确而陷入局部最优


3. 方法：适用场景——计算效率——实现复杂度
- 网格搜索：低维离散参数空间——低——低
- 随机搜索：中高维参数空间，计算资源有限——中——中
- 贝叶斯优化：高维连续参数空间，评估成本高——高——高

In [ ]:
# 如果数据集不平衡严重，可以考虑：
# 使用 SMOTE 进行过采样
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(random_state=42)
# X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# print(f"平衡后的类别分布:\n{pd.Series(y_train_balanced).value_counts()}")

In [ ]:
# =============================================================================
# 超参数优化方式2————贝叶斯超参数优化多模型训练与评估
# 参考：https://mp.weixin.qq.com/s/lvkn2PKv8AobZAsdpF9xjg
# 一、环境准备与可视化配置
# 首先，我们导入所需的库，并设置 Nature 风格配色 与绘图参数，以保证输出结果在学术论文与展示中具有顶刊水准。
# 这样配置后，模型结果图（如ROC曲线、混淆矩阵）会更符合国际期刊的展示风格。
# =============================================================================
# pip install scikit-optimize-0.10.2(pyaml-25.7.0)

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import numpy as np
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import shap
import warnings
import os
warnings.filterwarnings('ignore')

# 设置顶刊配色与字体
nature_colors = ['#E64B35', '#4DBBD5', '#00A087', '#3C5488', '#F39B7F']
sns.set_palette(nature_colors)
sns.set_style("whitegrid", {'grid.linestyle': '--', 'axes.edgecolor': '0.3'})
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['savefig.format'] = 'jpeg'

'''
# =============================================================================
# 参考：https://cloud.baidu.com/article/5601757
# =============================================================================
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
# 定义参数搜索空间
search_spaces = {
    'C': Real(0.1, 10, prior='log-uniform'),
    'gamma': Real(1e-4, 1e-1, prior='log-uniform'),
    'kernel': Categorical(['rbf', 'poly']),
    'degree': Integer(2, 5)
}
# 创建贝叶斯优化对象（评估50次）
bayes_search = BayesSearchCV(
    SVC(), search_spaces, n_iter=50, cv=5, scoring='accuracy'
)
bayes_search.fit(X_train, y_train)
print("最优参数:", bayes_search.best_params_)
'''

# =============================================================================
# 📂 二、数据准备与划分
# 读取样本数据，提取特征和标签，并划分训练集与测试集。同时，将数据集导出，以便后续分析。
# 此步骤保证数据划分的独立性与可追溯性，避免数据结果不可复现。
# =============================================================================
print("=" * 70)
print("步骤1: 使用项目预处理后的数据")
print("=" * 70)

# 保存结果到 CSV
output_dir = '/home/phl/PHL/Car-T/model_v1/output/bayes_cv_results/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 使用已经准备好的训练集和测试集数据（来自单元格8的数据预处理），确保与单元格12保持一致的数据格式和预处理管道
# X_train, y_train: 训练集特征和标签
# X_test, y_test: 测试集特征和标签
# preprocessor: 数据预处理管道
# cv_folds_indices: 3折交叉验证索引

print(f"训练集大小: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"测试集大小: X_test {X_test.shape}, y_test {y_test.shape}")
print(f"交叉验证策略: 使用预定义的 cv_folds_indices ({len(cv_folds_indices)} 折)")
print(f"数据预处理: 使用 preprocessor 管道")

# 导出训练集和测试集供后续分析
train_data_export = X_train.copy()
train_data_export['label'] = y_train
train_data_export.to_csv(output_dir + 'train_dataset_bayes.csv', index=False)

test_data_export = X_test.copy()
test_data_export['label'] = y_test
test_data_export.to_csv(output_dir + 'test_dataset_bayes.csv', index=False)
print("\n数据导出完成: train_dataset_bayes.csv, test_dataset_bayes.csv")

# =============================================================================
# 🔍 三、定义模型及其贝叶斯优化搜索空间
# 为了防止模型过拟合，我们在随机森林的参数空间中设置了较为保守的范围，并使用 BayesSearchCV 自动搜索最佳组合。
# 这里特别加入了 max_samples 与 ccp_alpha（复杂度剪枝），进一步提升模型的泛化性能。
# 参考单元格12的model_params结构，为多个模型定义贝叶斯优化的搜索空间
# =============================================================================
print("\n" + "=" * 70)
print("步骤2: 定义模型及其贝叶斯优化搜索空间")
print("=" * 70)

# 定义所有模型的参数空间（使用贝叶斯优化）
model_params_bayes = {
    'SVM': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', SVC(probability=True, random_state=42))
        ]),
        'params': {
            'classifier__C': Real(0.1, 10, prior='log-uniform'),
            'classifier__kernel': Categorical(['rbf', 'linear'])
        }
    },
    'XGBoost': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', xgb.XGBClassifier(
                objective='binary:logistic', 
                random_state=42,
                n_jobs=1
            ))
        ]),
        'params': {
            'classifier__learning_rate': Real(0.01, 0.3, prior='log-uniform'),
            'classifier__max_depth': Integer(3, 10),
            'classifier__n_estimators': Integer(50, 300)
        }
    },
    'LightGBM': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', lgb.LGBMClassifier(
                objective='binary', 
                metric='binary_logloss', 
                verbose=-1, 
                random_state=42, 
                n_jobs=1
            ))
        ]),
        'params': {
            'classifier__learning_rate': Real(0.01, 0.3, prior='log-uniform'),
            'classifier__n_estimators': Integer(50, 300),
            'classifier__num_leaves': Integer(20, 100)
        }
    },
    'Logistic Regression': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', LogisticRegression(random_state=42, max_iter=1000))
        ]),
        'params': {
            'classifier__C': Real(0.01, 100, prior='log-uniform'),
            'classifier__solver': Categorical(['liblinear', 'lbfgs'])
        }
    },
    'Random Forest': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1))
        ]),
        'params': {
            'classifier__n_estimators': Integer(50, 200),     
            'classifier__max_depth': Integer(3, 15),          
            'classifier__min_samples_split': Integer(10, 30), 
            'classifier__min_samples_leaf': Integer(5, 20),   
            'classifier__max_features': Categorical(['sqrt', 'log2']),
            'classifier__bootstrap': Categorical([True]),     
            'classifier__max_samples': Real(0.6, 0.9),        
            'classifier__ccp_alpha': Real(0.0, 0.01)
        }
    },
    'MLP': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('classifier', MLPClassifier(max_iter=500, random_state=42))
        ]),
        'params': {
            'classifier__hidden_layer_sizes': Categorical([    
                                            (50,), (100,), (150,),      # 单层网络
                                            (50, 50), (100, 50),        # 双层网络
                                            (100, 50, 25)]),            # 三层网络
            'classifier__alpha': Real(0.0001, 0.01, prior='log-uniform'),
            'classifier__learning_rate_init': Real(0.001, 0.01, prior='log-uniform')
        }
    },
    'KNN': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', KNeighborsClassifier())
        ]),
        'params': {
            # 交叉验证的某个 fold 中样本数太少（只有5个样本），n_neighbors=8 超过了该 fold 的样本数量
            'classifier__n_neighbors': Integer(1, 3),   
            'classifier__weights': Categorical(['uniform', 'distance']),
            'classifier__metric': Categorical(['euclidean', 'manhattan'])
        }
    },
    'AdaBoost': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', AdaBoostClassifier(algorithm='SAMME', random_state=42))
        ]),
        'params': {
            'classifier__n_estimators': Integer(50, 200),
            'classifier__learning_rate': Real(0.01, 2.0, prior='log-uniform')
        }
    },
    'Decision Tree': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', DecisionTreeClassifier(random_state=42))
        ]),
        'params': {
            'classifier__max_depth': Integer(3, 30),
            'classifier__min_samples_split': Integer(2, 20),
            'classifier__min_samples_leaf': Integer(1, 10)
        }
    },
    'Naive Bayes': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', GaussianNB())
        ]),
        'params': {
            'classifier__var_smoothing': Real(1e-10, 1e-7, prior='log-uniform')
        }
    },
    'Gradient Boosting': {
        'model': Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)), 
            ('classifier', GradientBoostingClassifier(random_state=42))
        ]),
        'params': {
            'classifier__n_estimators': Integer(50, 300),
            'classifier__learning_rate': Real(0.01, 0.3, prior='log-uniform'),
            'classifier__max_depth': Integer(3, 10),
            'classifier__subsample': Real(0.6, 1.0)
        }
    },
}

print(f"已定义 {len(model_params_bayes)} 个模型的贝叶斯优化搜索空间")
print(f"模型列表: {list(model_params_bayes.keys())}")

# =============================================================================
# ⚡ 四、BayesSearchCV超参数优化（批量处理多个模型）
# 通过贝叶斯优化+交叉验证，我们在合理的迭代次数内找到最佳参数组合，并保存优化过程。
# 这一步让随机森林在参数调优过程中避免了盲目搜索，提升了效率和模型的鲁棒性。
# 使用与单元格12一致的交叉验证策略（cv_folds_indices）
# refit=True 确保找到最佳参数后自动在完整训练集上重新训练
# =============================================================================
print("\n" + "=" * 70)
print("步骤3: 执行贝叶斯超参数优化（批量处理）")
print("=" * 70)

print(f"✅ 开始超参数调优与最终模型训练...")
print(f"使用数据: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"交叉验证策略: 使用预定义的 cv_folds_indices ({len(cv_folds_indices)} 折)")
print(f"优化方法: BayesSearchCV (n_iter=10)")

# 存储优化后的结果
final_models_bayes = {}      # 存储重训后的最佳模型
cv_results_data_bayes = []   # 存储结果数据
optimization_details = []    # 存储详细优化信息

# 遍历每个模型进行贝叶斯优化
for idx, (model_name, config) in enumerate(model_params_bayes.items(), 1):
    print(f"\n{'='*70}")
    print(f"[{idx}/{len(model_params_bayes)}] 正在优化 {model_name}...")
    print(f"{'='*70}")
    
    try:
        # 执行BayesSearchCV
        bayes_search = BayesSearchCV(
            estimator=config['model'],
            search_spaces=config['params'],
            n_iter=10,  # 贝叶斯优化迭代次数
            cv=cv_folds_indices,  # 使用与单元格12一致的交叉验证
            scoring='roc_auc',
            n_jobs=-1,
            random_state=42,
            verbose=0,  # 减少输出
            refit=True  # 自动使用最佳参数在完整训练集上重新训练
        )
        
        print(f"  ⏳ 开始贝叶斯优化...")
        bayes_search.fit(X_train, y_train)
        
        # 1. 保存最佳模型（已在全量数据上训练好）
        final_models_bayes[model_name] = bayes_search.best_estimator_
        
        # 2. 记录结果
        best_score = bayes_search.best_score_
        best_params = bayes_search.best_params_
        best_std = bayes_search.cv_results_['std_test_score'][bayes_search.best_index_]
        
        print(f"  ✅ 优化完成!")
        print(f"  -> 最佳 AUC: {best_score:.4f} (±{best_std:.4f})")
        print(f"  -> 最佳参数: {best_params}")
        
        # 收集每一折的详细分数
        fold_scores = []
        for i in range(len(cv_folds_indices)):
            fold_scores.append(bayes_search.cv_results_[f'split{i}_test_score'][bayes_search.best_index_])
        
        cv_results_data_bayes.append({
            'Model': model_name,
            'Mean AUC': best_score,
            'Std AUC': best_std,
            'Scores': fold_scores,
            'Best Params': str(best_params)
        })
        
        optimization_details.append({
            'Model': model_name,
            'Status': 'Success',
            'Best AUC': f"{best_score:.4f}",
            'Std': f"{best_std:.4f}",
            'Best Params': str(best_params)
        })
        
        # 保存单个模型
        model_filename = f"{model_name.replace(' ', '_')}_bayes_optimized.pkl"
        output_dir_bayes = os.path.join(output_dir, 'models')
        if not os.path.exists(output_dir_bayes):
            os.makedirs(output_dir_bayes)
        joblib.dump(bayes_search.best_estimator_, os.path.join(output_dir_bayes, model_filename))
        print(f"  💾 模型已保存: {model_filename}")
        
    except Exception as e:
        print(f"  ❌ {model_name} 优化失败: {str(e)}")
        optimization_details.append({
            'Model': model_name,
            'Status': 'Failed',
            'Best AUC': 'N/A',
            'Std': 'N/A',
            'Best Params': f'Error: {str(e)}'
        })
        continue

# =============================================================================
# 📊 五、模型性能评估（批量处理）
# 使用最佳参数训练模型，并在训练集与测试集上分别计算准确率、精确率、召回率、F1、ROC AUC等性能指标，同时进行过拟合分析。
# 通过对比训练集与测试集表现，可以直观判断模型是否存在过拟合。
# =============================================================================
print("\n" + "=" * 70)
print("步骤4: 模型性能评估（批量处理）")
print("=" * 70)

performance_results = []

for model_name, model in final_models_bayes.items():
    print(f"\n评估 {model_name}...")
    
    try:
        # 训练集评估
        y_train_pred = model.predict(X_train)
        y_train_pred_proba = model.predict_proba(X_train)[:, 1]
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_precision = precision_score(y_train, y_train_pred, zero_division=0)
        train_recall = recall_score(y_train, y_train_pred, zero_division=0)
        train_f1 = f1_score(y_train, y_train_pred, zero_division=0)
        train_roc_auc = roc_auc_score(y_train, y_train_pred_proba)
        
        # 测试集评估
        y_test_pred = model.predict(X_test)
        y_test_pred_proba = model.predict_proba(X_test)[:, 1]
        test_accuracy = accuracy_score(y_test, y_test_pred)
        test_precision = precision_score(y_test, y_test_pred, zero_division=0)
        test_recall = recall_score(y_test, y_test_pred, zero_division=0)
        test_f1 = f1_score(y_test, y_test_pred, zero_division=0)
        test_roc_auc = roc_auc_score(y_test, y_test_pred_proba)
        
        performance_results.append({
            'Model': model_name,
            'Train_Accuracy': train_accuracy,
            'Train_Precision': train_precision,
            'Train_Recall': train_recall,
            'Train_F1': train_f1,
            'Train_ROC_AUC': train_roc_auc,
            'Test_Accuracy': test_accuracy,
            'Test_Precision': test_precision,
            'Test_Recall': test_recall,
            'Test_F1': test_f1,
            'Test_ROC_AUC': test_roc_auc,
            'AUC_Diff': abs(train_roc_auc - test_roc_auc)
        })
        
        print(f"  训练集 - Acc: {train_accuracy:.4f}, Prec: {train_precision:.4f}, Rec: {train_recall:.4f}, F1: {train_f1:.4f}, AUC: {train_roc_auc:.4f}")
        print(f"  测试集 - Acc: {test_accuracy:.4f}, Prec: {test_precision:.4f}, Rec: {test_recall:.4f}, F1: {test_f1:.4f}, AUC: {test_roc_auc:.4f}")
        print(f"  过拟合分析: AUC差异 = {abs(train_roc_auc - test_roc_auc):.4f}")
        
    except Exception as e:
        print(f"  ❌ {model_name} 评估失败: {str(e)}")
        continue

# =============================================================================
# 📈 六、结果汇总与可视化：ROC与混淆矩阵
# 绘制训练集与测试集的 ROC曲线，并输出混淆矩阵，全面展示模型分类效果。
# =============================================================================
print("\n" + "=" * 70)
print("步骤5: 结果汇总与可视化")
print("=" * 70)

# 将结果转换为 DataFrame
df_cv_results = pd.DataFrame(cv_results_data_bayes).sort_values('Mean AUC', ascending=False)
df_performance = pd.DataFrame(performance_results).sort_values('Test_ROC_AUC', ascending=False)

print("\n交叉验证优化结果汇总:")
print(df_cv_results[['Model', 'Mean AUC', 'Std AUC']].to_string(index=False))

print("\n测试集性能汇总:")
print(df_performance[['Model', 'Test_Accuracy', 'Test_Precision', 'Test_Recall', 'Test_F1', 'Test_ROC_AUC', 'AUC_Diff']].to_string(index=False))

df_cv_results.drop(columns=['Scores']).to_csv(os.path.join(output_dir, 'bayes_cv_optimization_results.csv'), index=False)
df_performance.to_csv(os.path.join(output_dir, 'bayes_performance_results.csv'), index=False)

print(f"\n结果已保存到: {output_dir}")

# 可视化：交叉验证性能对比
fig, ax = plt.subplots(figsize=(14, 7))
models_list = df_cv_results['Model'].tolist()
means = df_cv_results['Mean AUC'].tolist()
stds = df_cv_results['Std AUC'].tolist()
x_pos = np.arange(len(models_list))

ax.bar(x_pos, means, yerr=stds, align='center', alpha=0.7, 
       capsize=10, color=nature_colors[:len(models_list)])
ax.set_ylabel('AUC Score (CV Optimized)', fontsize=12)
ax.set_title('Cross-Validation Performance after Bayesian Optimization', fontsize=14)
ax.set_xticks(x_pos)
ax.set_xticklabels(models_list, rotation=45, ha='right')
ax.set_ylim([0.5, 1.0])
ax.yaxis.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'bayes_cv_optimized_barplot.png'), dpi=300)
plt.show()

print("✅ 可视化图表已保存: bayes_cv_optimized_barplot.png")

# =============================================================================
# 七、将最佳模型赋值给全局变量（保持与单元格12的一致性）
# 这样后续单元格直接使用 svm, rf 等变量时，用的就是调优后的最佳模型
# =============================================================================
print("\n" + "=" * 70)
print("步骤6: 更新全局变量")
print("=" * 70)

# 将优化后的模型赋值给全局变量
if 'SVM' in final_models_bayes:
    svm = final_models_bayes['SVM']
    print("✅ svm 已更新")
if 'XGBoost' in final_models_bayes:
    xgb_model = final_models_bayes['XGBoost']
    print("✅ xgb_model 已更新")
if 'LightGBM' in final_models_bayes:
    lgb_model = final_models_bayes['LightGBM']
    print("✅ lgb_model 已更新")
if 'Logistic Regression' in final_models_bayes:
    log_reg = final_models_bayes['Logistic Regression']
    print("✅ log_reg 已更新")
if 'Random Forest' in final_models_bayes:
    rf = final_models_bayes['Random Forest']
    print("✅ rf 已更新")
if 'MLP' in final_models_bayes:
    mlp = final_models_bayes['MLP']
    print("✅ mlp 已更新")
if 'KNN' in final_models_bayes:
    knn = final_models_bayes['KNN']
    print("✅ knn 已更新")
if 'AdaBoost' in final_models_bayes:
    ada = final_models_bayes['AdaBoost']
    print("✅ ada 已更新")
if 'Decision Tree' in final_models_bayes:
    dt = final_models_bayes['Decision Tree']
    print("✅ dt 已更新")
if 'Naive Bayes' in final_models_bayes:
    nb = final_models_bayes['Naive Bayes']
    print("✅ nb 已更新")
if 'Gradient Boosting' in final_models_bayes:
    # 注意：Gradient Boosting 通常不使用缩写，但为了一致性，可以创建一个变量
    gb = final_models_bayes['Gradient Boosting']
    print("✅ gb 已更新")

# 为了与单元格12保持完全一致，也创建final_models字典
final_models = final_models_bayes.copy()
print("\n✅ final_models 字典已更新，包含所有优化后的模型")

# =============================================================================
# 八、总结报告
# =============================================================================
print("\n" + "=" * 70)
print("完成! 贝叶斯优化多模型训练总结")
print("=" * 70)
print(f"\n1. 成功优化模型数量: {len(final_models_bayes)}/{len(model_params_bayes)}")
print(f"2. 最佳模型（按测试集AUC排序）:")
if len(df_performance) > 0:
    for idx, row in df_performance.head(5).iterrows():
        print(f"   {row['Model']}: Test AUC = {row['Test_ROC_AUC']:.4f}")
print(f"\n3. 输出文件:")
print(f"   - 训练数据: train_dataset_bayes.csv")
print(f"   - 测试数据: test_dataset_bayes.csv")
print(f"   - 交叉验证结果: {output_dir}/bayes_cv_optimization_results.csv")
print(f"   - 性能评估结果: {output_dir}/bayes_performance_results.csv")
print(f"   - 可视化图表: {output_dir}/bayes_cv_optimized_barplot.png")
print(f"\n4. 全局变量已更新，可以直接用于后续ROC分析和SHAP分析")
print(f"   - final_models: 包含所有优化后的模型")
print(f"   - 单独变量: svm, xgb_model, lgb_model, log_reg, rf, mlp, knn, ada, dt, nb, gb")
print("\n" + "=" * 70)


# 🌲 七、特征重要性与解释性分析（SHAP）
# 为了理解模型决策依据，我们不仅计算了随机森林自带的重要性，还结合 SHAP值 提供更细致的解释。

In [ ]:
# =============================================================================
# 超参数优化方式3————Optuna超参数优化多模型训练与评估
# 使用 Optuna 替代 BayesSearchCV，实现更灵活的超参数优化
# =============================================================================
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import shap
import warnings
import os
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
warnings.filterwarnings('ignore')

# 设置顶刊配色与字体
nature_colors = ['#E64B35', '#4DBBD5', '#00A087', '#3C5488', '#F39B7F']
sns.set_palette(nature_colors)
sns.set_style("whitegrid", {'grid.linestyle': '--', 'axes.edgecolor': '0.3'})
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['savefig.format'] = 'jpeg'

# 禁用Optuna的日志输出
optuna.logging.set_verbosity(optuna.logging.WARNING)

'''
# =============================================================================
# Optuna超参数优化XGBoost模型
# 参考：https://zhuanlan.zhihu.com/p/1964684051261666199
# =============================================================================

import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_breast_cancer

# 加载示例数据集
X, y = load_breast_cancer(return_X_y=True)

def objective(trial):
    # 定义超参数搜索空间
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
    }

    # 创建并评估模型
    model = xgb.XGBClassifier(**params, random_state=42)
    score = cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()

    return score

# 创建 study 对象并开始优化
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("最佳参数:", study.best_params)
print("最佳准确率:", study.best_value)
'''

# =============================================================================
# 📂 二、数据准备与划分
# =============================================================================
print("=" * 70)
print("步骤1: 使用项目预处理后的数据")
print("=" * 70)

# 保存结果到 CSV
output_dir = '/home/phl/PHL/Car-T/model_v1/output/optuna_cv_results/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"训练集大小: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"测试集大小: X_test {X_test.shape}, y_test {y_test.shape}")
print(f"交叉验证策略: 使用预定义的 cv_folds_indices ({len(cv_folds_indices)} 折)")
print(f"数据预处理: 使用 preprocessor 管道")

# 导出训练集和测试集供后续分析
train_data_export = X_train.copy()
train_data_export['label'] = y_train
train_data_export.to_csv(output_dir + 'train_dataset_optuna.csv', index=False)

test_data_export = X_test.copy()
test_data_export['label'] = y_test
test_data_export.to_csv(output_dir + 'test_dataset_optuna.csv', index=False)
print("\n数据导出完成: train_dataset_optuna.csv, test_dataset_optuna.csv")

# =============================================================================
# 🔍 三、定义Optuna优化函数
# 为每个模型定义objective函数，使用连续搜索空间
# =============================================================================
print("\n" + "=" * 70)
print("步骤2: 定义 Optuna 超参数优化函数")
print("=" * 70)

# 创建自定义交叉验证评分器
def custom_cv_score(model, X, y, cv_indices):
    """使用预定义的cv_folds_indices进行交叉验证"""
    scores = []
    for train_idx, val_idx in cv_indices:
        X_train_fold = X.iloc[train_idx]
        y_train_fold = y.iloc[train_idx]
        X_val_fold = X.iloc[val_idx]
        y_val_fold = y.iloc[val_idx]
        
        model.fit(X_train_fold, y_train_fold)
        y_pred_proba = model.predict_proba(X_val_fold)[:, 1]
        score = roc_auc_score(y_val_fold, y_pred_proba)
        scores.append(score)
    
    return np.mean(scores)

# 定义各模型的Optuna优化函数
def objective_catboost(trial):
    """CatBoost优化函数"""
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'iterations': trial.suggest_int('iterations', 50, 300),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_state': 42,
        'verbose': False,
        'thread_count': 1
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', CatBoostClassifier(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_svm(trial):
    """SVM优化函数"""
    params = {
        'C': trial.suggest_float('C', 0.1, 10, log=True),
        'kernel': trial.suggest_categorical('kernel', ['rbf', 'linear']),
        'probability': True,
        'random_state': 42
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', SVC(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_xgboost(trial):
    """XGBoost优化函数"""
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'objective': 'binary:logistic',
        'random_state': 42,
        'n_jobs': 1
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', xgb.XGBClassifier(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_lightgbm(trial):
    """LightGBM优化函数"""
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbose': -1,
        'random_state': 42,
        'n_jobs': 1
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', lgb.LGBMClassifier(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_logistic(trial):
    """Logistic Regression优化函数"""
    params = {
        'C': trial.suggest_float('C', 0.01, 100, log=True),
        'solver': trial.suggest_categorical('solver', ['liblinear', 'lbfgs']),
        'max_iter': 1000,
        'random_state': 42
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', LogisticRegression(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_random_forest(trial):
    """Random Forest优化函数"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_samples_split': trial.suggest_int('min_samples_split', 10, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 5, 20),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': True,
        'max_samples': trial.suggest_float('max_samples', 0.6, 0.9),
        'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 0.01),
        'random_state': 42,
        'n_jobs': -1
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', RandomForestClassifier(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_mlp(trial):
    """MLP优化函数"""
    # 定义隐藏层结构
    hidden_layer_choice = trial.suggest_categorical('hidden_layer_choice', [
        'single_50', 'single_100', 'single_150',
        'double_50_50', 'double_100_50',
        'triple_100_50_25'
    ])
    
    hidden_layers = {
        'single_50': (50,),
        'single_100': (100,),
        'single_150': (150,),
        'double_50_50': (50, 50),
        'double_100_50': (100, 50),
        'triple_100_50_25': (100, 50, 25)
    }
    
    params = {
        'hidden_layer_sizes': hidden_layers[hidden_layer_choice],
        'alpha': trial.suggest_float('alpha', 0.0001, 0.01, log=True),
        'learning_rate_init': trial.suggest_float('learning_rate_init', 0.001, 0.01, log=True),
        'max_iter': 500,
        'random_state': 42
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', MLPClassifier(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_knn(trial):
    """KNN优化函数"""
    params = {
        'n_neighbors': trial.suggest_int('n_neighbors', 1, 5),
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
        'metric': trial.suggest_categorical('metric', ['euclidean', 'manhattan'])
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', KNeighborsClassifier(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_adaboost(trial):
    """AdaBoost优化函数"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 2.0, log=True),
        'algorithm': 'SAMME',
        'random_state': 42
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', AdaBoostClassifier(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_decision_tree(trial):
    """Decision Tree优化函数"""
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'random_state': 42
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', DecisionTreeClassifier(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_naive_bayes(trial):
    """Naive Bayes优化函数"""
    params = {
        'var_smoothing': trial.suggest_float('var_smoothing', 1e-10, 1e-7, log=True)
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', GaussianNB(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

def objective_gradient_boosting(trial):
    """Gradient Boosting优化函数"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'random_state': 42
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
        ('classifier', GradientBoostingClassifier(**params))
    ])
    
    return custom_cv_score(model, X_train, y_train, cv_folds_indices)

# 定义模型配置字典
model_configs = {
    'CatBoost': {
        'objective_func': objective_catboost,
        'model_class': CatBoostClassifier
    },
    'SVM': {
        'objective_func': objective_svm,
        'model_class': SVC
    },
    'XGBoost': {
        'objective_func': objective_xgboost,
        'model_class': xgb.XGBClassifier
    },
    'LightGBM': {
        'objective_func': objective_lightgbm,
        'model_class': lgb.LGBMClassifier
    },
    'Logistic Regression': {
        'objective_func': objective_logistic,
        'model_class': LogisticRegression
    },
    'Random Forest': {
        'objective_func': objective_random_forest,
        'model_class': RandomForestClassifier
    },
    'MLP': {
        'objective_func': objective_mlp,
        'model_class': MLPClassifier
    },
    'KNN': {
        'objective_func': objective_knn,
        'model_class': KNeighborsClassifier
    },
    'AdaBoost': {
        'objective_func': objective_adaboost,
        'model_class': AdaBoostClassifier
    },
    'Decision Tree': {
        'objective_func': objective_decision_tree,
        'model_class': DecisionTreeClassifier
    },
    'Naive Bayes': {
        'objective_func': objective_naive_bayes,
        'model_class': GaussianNB
    },
    'Gradient Boosting': {
        'objective_func': objective_gradient_boosting,
        'model_class': GradientBoostingClassifier
    }
}

print(f"已定义 {len(model_configs)} 个模型的 Optuna 优化函数")
print(f"模型列表: {list(model_configs.keys())}")

# =============================================================================
# ⚡ 四、Optuna超参数优化（批量处理多个模型）
# =============================================================================
print("\n" + "=" * 70)
print("步骤3: 执行 Optuna 超参数优化（批量处理）")
print("=" * 70)

print(f"✅ 开始超参数调优与最终模型训练...")
print(f"使用数据: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"交叉验证策略: 使用预定义的 cv_folds_indices ({len(cv_folds_indices)} 折)")
print(f"优化方法: Optuna (n_trials=50)")

# 存储优化后的结果
final_models = {}           # 存储最佳模型
cv_results_data = []        # 存储结果数据
optimization_details = []   # 存储详细优化信息
all_trials_data = []        # 存储所有trial信息

# 遍历每个模型进行Optuna优化
for idx, (model_name, config) in enumerate(model_configs.items(), 1):
    print(f"\n{'='*70}")
    print(f"[{idx}/{len(model_configs)}] 正在优化 {model_name}...")
    print(f"{'='*70}")
    
    try:
        # 创建Optuna study
        study = optuna.create_study(
            direction='maximize',
            study_name=f'{model_name}_optimization',
            sampler=optuna.samplers.TPESampler(seed=42)
        )
        
        print(f"  ⏳ 开始 Optuna 优化 (50 trials)...")
        
        # 执行优化
        study.optimize(config['objective_func'], n_trials=50, show_progress_bar=False)
        
        # 获取最佳参数
        best_params = study.best_params
        best_score = study.best_value
        
        # 记录所有trials信息
        for trial in study.trials:
            all_trials_data.append({
                'Model': model_name,
                'Trial': trial.number,
                'AUC': trial.value if trial.value is not None else 'Failed',
                'Params': str(trial.params)
            })
        
        print(f"  ✅ 优化完成!")
        print(f"  -> 最佳 AUC: {best_score:.4f}")
        print(f"  -> 最佳参数: {best_params}")
        
        # 使用最佳参数重新训练模型
        print(f"  ⏳ 使用最佳参数在完整训练集上训练...")
        
        # 根据模型类型构建最终模型
        if model_name == 'MLP':
            # MLP需要特殊处理hidden_layer_sizes
            hidden_layers = {
                'single_50': (50,),
                'single_100': (100,),
                'single_150': (150,),
                'double_50_50': (50, 50),
                'double_100_50': (100, 50),
                'triple_100_50_25': (100, 50, 25)
            }
            final_params = {k: v for k, v in best_params.items() if k != 'hidden_layer_choice'}
            final_params['hidden_layer_sizes'] = hidden_layers[best_params['hidden_layer_choice']]
            final_params['max_iter'] = 500
            final_params['random_state'] = 42
        elif model_name == 'CatBoost':
            final_params = best_params.copy()
            final_params['random_state'] = 42
            final_params['verbose'] = False
            final_params['thread_count'] = 1
        elif model_name == 'XGBoost':
            final_params = best_params.copy()
            final_params['objective'] = 'binary:logistic'
            final_params['random_state'] = 42
            final_params['n_jobs'] = 1
        elif model_name == 'LightGBM':
            final_params = best_params.copy()
            final_params['objective'] = 'binary'
            final_params['metric'] = 'binary_logloss'
            final_params['verbose'] = -1
            final_params['random_state'] = 42
            final_params['n_jobs'] = 1
        elif model_name == 'Logistic Regression':
            final_params = best_params.copy()
            final_params['max_iter'] = 1000
            final_params['random_state'] = 42
        elif model_name == 'Random Forest':
            final_params = best_params.copy()
            final_params['bootstrap'] = True
            final_params['random_state'] = 42
            final_params['n_jobs'] = -1
        elif model_name == 'SVM':
            final_params = best_params.copy()
            final_params['probability'] = True
            final_params['random_state'] = 42
        elif model_name == 'AdaBoost':
            final_params = best_params.copy()
            final_params['algorithm'] = 'SAMME'
            final_params['random_state'] = 42
        elif model_name in ['Decision Tree', 'Gradient Boosting']:
            final_params = best_params.copy()
            final_params['random_state'] = 42
        else:
            final_params = best_params.copy()
        
        # 创建最终模型
        final_model = Pipeline([
            ('preprocessor', preprocessor),
            ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('classifier', config['model_class'](**final_params))
        ])
        
        # 在完整训练集上训练
        final_model.fit(X_train, y_train)
        
        # 保存模型
        final_models[model_name] = final_model
        
        # 计算交叉验证的标准差（使用最佳参数重新评估）
        fold_scores = []
        for train_idx, val_idx in cv_folds_indices:
            X_train_fold = X_train.iloc[train_idx]
            y_train_fold = y_train.iloc[train_idx]
            X_val_fold = X_train.iloc[val_idx]
            y_val_fold = y_train.iloc[val_idx]
            
            fold_model = Pipeline([
                ('preprocessor', preprocessor),
                ('final_imputer', SimpleImputer(strategy='constant', fill_value=0)),
                ('classifier', config['model_class'](**final_params))
            ])
            fold_model.fit(X_train_fold, y_train_fold)
            y_pred_proba = fold_model.predict_proba(X_val_fold)[:, 1]
            score = roc_auc_score(y_val_fold, y_pred_proba)
            fold_scores.append(score)
        
        best_std = np.std(fold_scores)
        
        cv_results_data.append({
            'Model': model_name,
            'Mean AUC': best_score,
            'Std AUC': best_std,
            'Scores': fold_scores,
            'Best Params': str(best_params),
            'N_Trials': len(study.trials)
        })
        
        optimization_details.append({
            'Model': model_name,
            'Status': 'Success',
            'Best AUC': f"{best_score:.4f}",
            'Std': f"{best_std:.4f}",
            'Best Params': str(best_params),
            'N_Trials': len(study.trials)
        })
        
        # 保存单个模型
        model_filename = f"{model_name.replace(' ', '_')}_optuna_optimized.pkl"
        output_dir_models = os.path.join(output_dir, 'models')
        if not os.path.exists(output_dir_models):
            os.makedirs(output_dir_models)
        joblib.dump(final_model, os.path.join(output_dir_models, model_filename))
        print(f"  💾 模型已保存: {model_filename}")
        
    except Exception as e:
        print(f"  ❌ {model_name} 优化失败: {str(e)}")
        optimization_details.append({
            'Model': model_name,
            'Status': 'Failed',
            'Best AUC': 'N/A',
            'Std': 'N/A',
            'Best Params': f'Error: {str(e)}',
            'N_Trials': 0
        })
        continue

# =============================================================================
# 📊 五、模型性能评估（批量处理）
# =============================================================================
print("\n" + "=" * 70)
print("步骤4: 模型性能评估（批量处理）")
print("=" * 70)

performance_results = []

for model_name, model in final_models.items():
    print(f"\n评估 {model_name}...")
    
    try:
        # 训练集评估
        y_train_pred = model.predict(X_train)
        y_train_pred_proba = model.predict_proba(X_train)[:, 1]
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_precision = precision_score(y_train, y_train_pred, zero_division=0)
        train_recall = recall_score(y_train, y_train_pred, zero_division=0)
        train_f1 = f1_score(y_train, y_train_pred, zero_division=0)
        train_roc_auc = roc_auc_score(y_train, y_train_pred_proba)
        
        # 测试集评估
        y_test_pred = model.predict(X_test)
        y_test_pred_proba = model.predict_proba(X_test)[:, 1]
        test_accuracy = accuracy_score(y_test, y_test_pred)
        test_precision = precision_score(y_test, y_test_pred, zero_division=0)
        test_recall = recall_score(y_test, y_test_pred, zero_division=0)
        test_f1 = f1_score(y_test, y_test_pred, zero_division=0)
        test_roc_auc = roc_auc_score(y_test, y_test_pred_proba)
        
        performance_results.append({
            'Model': model_name,
            'Train_Accuracy': train_accuracy,
            'Train_Precision': train_precision,
            'Train_Recall': train_recall,
            'Train_F1': train_f1,
            'Train_ROC_AUC': train_roc_auc,
            'Test_Accuracy': test_accuracy,
            'Test_Precision': test_precision,
            'Test_Recall': test_recall,
            'Test_F1': test_f1,
            'Test_ROC_AUC': test_roc_auc,
            'AUC_Diff': abs(train_roc_auc - test_roc_auc)
        })
        
        print(f"  训练集 - Acc: {train_accuracy:.4f}, Prec: {train_precision:.4f}, Rec: {train_recall:.4f}, F1: {train_f1:.4f}, AUC: {train_roc_auc:.4f}")
        print(f"  测试集 - Acc: {test_accuracy:.4f}, Prec: {test_precision:.4f}, Rec: {test_recall:.4f}, F1: {test_f1:.4f}, AUC: {test_roc_auc:.4f}")
        print(f"  过拟合分析: AUC差异 = {abs(train_roc_auc - test_roc_auc):.4f}")
        
    except Exception as e:
        print(f"  ❌ {model_name} 评估失败: {str(e)}")
        continue

# =============================================================================
# 📈 六、结果汇总与可视化
# =============================================================================
print("\n" + "=" * 70)
print("步骤5: 结果汇总与可视化")
print("=" * 70)

# 将结果转换为 DataFrame
df_cv_results = pd.DataFrame(cv_results_data).sort_values('Mean AUC', ascending=False)
df_performance = pd.DataFrame(performance_results).sort_values('Test_ROC_AUC', ascending=False)
df_all_trials = pd.DataFrame(all_trials_data)

print("\n交叉验证优化结果汇总:")
print(df_cv_results[['Model', 'Mean AUC', 'Std AUC', 'N_Trials']].to_string(index=False))

print("\n测试集性能汇总:")
print(df_performance[['Model', 'Test_Accuracy', 'Test_Precision', 'Test_Recall', 'Test_F1', 'Test_ROC_AUC', 'AUC_Diff']].to_string(index=False))

df_cv_results.drop(columns=['Scores']).to_csv(os.path.join(output_dir, 'optuna_cv_optimization_results.csv'), index=False)
df_performance.to_csv(os.path.join(output_dir, 'optuna_performance_results.csv'), index=False)
df_all_trials.to_csv(os.path.join(output_dir, 'optuna_all_trials.csv'), index=False)

print(f"\n结果已保存到: {output_dir}")

# 可视化：交叉验证性能对比
fig, ax = plt.subplots(figsize=(14, 7))
models_list = df_cv_results['Model'].tolist()
means = df_cv_results['Mean AUC'].tolist()
stds = df_cv_results['Std AUC'].tolist()
x_pos = np.arange(len(models_list))

ax.bar(x_pos, means, yerr=stds, align='center', alpha=0.7, 
       capsize=10, color=nature_colors[:len(models_list)])
ax.set_ylabel('AUC Score (CV Optimized)', fontsize=12)
ax.set_title('Cross-Validation Performance after Optuna Optimization', fontsize=14)
ax.set_xticks(x_pos)
ax.set_xticklabels(models_list, rotation=45, ha='right')
ax.set_ylim([0.5, 1.0])
ax.yaxis.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'optuna_cv_optimized_barplot.png'), dpi=300)
plt.show()

print("✅ 可视化图表已保存: optuna_cv_optimized_barplot.png")

# =============================================================================
# 七、将最佳模型赋值给全局变量（保持与后续单元格的兼容性）
# =============================================================================
print("\n" + "=" * 70)
print("步骤6: 更新全局变量")
print("=" * 70)

# 将优化后的模型赋值给全局变量
if 'CatBoost' in final_models:
    catboost_model = final_models['CatBoost']
    print("✅ catboost_model 已更新")
if 'SVM' in final_models:
    svm = final_models['SVM']
    print("✅ svm 已更新")
if 'XGBoost' in final_models:
    xgb_model = final_models['XGBoost']
    print("✅ xgb_model 已更新")
if 'LightGBM' in final_models:
    lgb_model = final_models['LightGBM']
    print("✅ lgb_model 已更新")
if 'Logistic Regression' in final_models:
    log_reg = final_models['Logistic Regression']
    print("✅ log_reg 已更新")
if 'Random Forest' in final_models:
    rf = final_models['Random Forest']
    print("✅ rf 已更新")
if 'MLP' in final_models:
    mlp = final_models['MLP']
    print("✅ mlp 已更新")
if 'KNN' in final_models:
    knn = final_models['KNN']
    print("✅ knn 已更新")
if 'AdaBoost' in final_models:
    ada = final_models['AdaBoost']
    print("✅ ada 已更新")
if 'Decision Tree' in final_models:
    dt = final_models['Decision Tree']
    print("✅ dt 已更新")
if 'Naive Bayes' in final_models:
    nb = final_models['Naive Bayes']
    print("✅ nb 已更新")
if 'Gradient Boosting' in final_models:
    gb = final_models['Gradient Boosting']
    print("✅ gb 已更新")

print("\n✅ final_models 字典已更新，包含所有优化后的模型")

# =============================================================================
# 八、总结报告
# =============================================================================
print("\n" + "=" * 70)
print("完成! Optuna 优化多模型训练总结")
print("=" * 70)
print(f"\n1. 成功优化模型数量: {len(final_models)}/{len(model_configs)}")
print(f"2. 最佳模型（按测试集AUC排序）:")
if len(df_performance) > 0:
    for idx, row in df_performance.head(5).iterrows():
        print(f"   {row['Model']}: Test AUC = {row['Test_ROC_AUC']:.4f}")
print(f"\n3. 输出文件:")
print(f"   - 训练数据: train_dataset_optuna.csv")
print(f"   - 测试数据: test_dataset_optuna.csv")
print(f"   - 交叉验证结果: {output_dir}/optuna_cv_optimization_results.csv")
print(f"   - 性能评估结果: {output_dir}/optuna_performance_results.csv")
print(f"   - 所有trial数据: {output_dir}/optuna_all_trials.csv")
print(f"   - 可视化图表: {output_dir}/optuna_cv_optimized_barplot.png")
print(f"\n4. 全局变量已更新，可以直接用于后续ROC分析和SHAP分析")
print(f"   - final_models: 包含所有优化后的模型")
print(f"   - 单独变量: catboost_model, svm, xgb_model, lgb_model, log_reg, rf, mlp, knn, ada, dt, nb, gb")
print("\n5. Optuna优化特点:")
print(f"   - 使用连续搜索空间，更精细的参数调优")
print(f"   - 每个模型运行50次trial，自适应采样策略")
print(f"   - 保存了所有trial的详细信息供分析")
print("\n" + "=" * 70)


In [ ]:
# 交叉验证 ROC 曲线绘制代码
## 1.plot_cv_roc_curve 函数：
# 接收一个基础模型（如 mlp），使用 sklearn.base.clone 复制它，确保每一折都是从头训练，互不干扰。
# 直接遍历你之前生成的 cv_folds 列表, 使用 .iloc[train_idx] 从 X_train 和 y_train 中提取每一折的数据。
# 计算并绘制每一折的 ROC 曲线，最后计算平均 AUC 和标准差区域。
## 2.模型兼容性：
# 此代码适用于 svm, rf, mlp, log_reg, xgb_model 等遵循 Scikit-learn 接口（有 .fit() 和 .predict_proba()）的模型。
# 如果是 lgb.train 出来的 Booster 对象则没有 fit/predict_proba 方法，这里假设传入的是 sklearn 接口的模型
# 注意：如果需要画 LightGBM，请使用 LGBMClassifier 包装器。对于 LightGBM (sklearn API) 和 XGBoost，通常都有 predict_proba
from sklearn.metrics import auc, roc_curve
from sklearn.base import clone
import matplotlib.pyplot as plt
import numpy as np

def plot_cv_roc_curve(base_model, X, y, cv_folds, model_name, ax=None, is_lgb=False):
    """
    使用预定义的 cv_folds 绘制交叉验证 ROC 曲线
    
    参数:
        base_model: 初始模型对象 (需支持 scikit-learn 接口)
        X: 训练集特征 (DataFrame)
        y: 训练集标签 (Series)
        cv_folds: 包含 (train_idx, val_idx) 元组的列表 (整数索引!)
        model_name: 模型名称字符串
        ax: matplotlib axes 对象 (可选)
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 8))

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    
    # 🆕 定义更鲜艳的颜色方案（使用饱和度更高的颜色）
    fold_colors = ["#F31414", "#956DF1", "#F37A00", "#1862EB", "#F1C100", "#11F5E6"]
    
    print(f"正在绘制 {model_name} 的 {len(cv_folds)} 折交叉验证 ROC 曲线...")

    # 遍历每一折
    for i, (train_idx, val_idx) in enumerate(cv_folds):
        print(f"  处理第 {i+1} 折: 训练集 {len(train_idx)} 样本, 验证集 {len(val_idx)} 样本")
        
        # 克隆模型
        model = clone(base_model)
        
        # 直接使用整数索引
        X_train_fold = X.iloc[train_idx]
        y_train_fold = y.iloc[train_idx]
        X_val_fold = X.iloc[val_idx]
        y_val_fold = y.iloc[val_idx]
        
        # 训练模型
        model.fit(X_train_fold, y_train_fold)
        
        # 预测概率
        if is_lgb:
            # y_prob = model.predict(X_val_fold, num_iteration=model.best_iteration)
            y_prob = model.predict(X_val_fold)
        else:
            y_prob = model.predict_proba(X_val_fold)[:, 1]
        
        # 计算 ROC 曲线
        fpr, tpr, _ = roc_curve(y_val_fold, y_prob)
        
        # 插值
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        
        # 计算 AUC
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        
        # 绘制当前折的曲线，提高透明度 (alpha 从 0.3 → 0.7)，使用鲜艳颜色
        ax.plot(fpr, tpr, lw=2, alpha=0.7,
                color=fold_colors[i % len(fold_colors)],
                label=f'ROC fold {i+1} (AUC = {roc_auc:.2f})')

    # 绘制对角线
    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Chance', alpha=.8)

    # 计算并绘制平均 ROC
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    
    # 使用更醒目的蓝色
    ax.plot(mean_fpr, mean_tpr, color='#1E3A8A',
            label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
            lw=3, alpha=.9)

    # 绘制标准差阴影
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                    label=r'$\pm$ 1 std. dev.')

    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
           title=f"ROC Curve - {model_name} ({len(cv_folds)}-Fold CV)")
    ax.legend(loc="lower right", fontsize=9)
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.grid(alpha=0.3)
    
    return ax

# ========== 准备数据 ==========
# 使用 X_train (已预处理的特征) 和 y_train (标签)
y_train_series = y_train.reset_index(drop=True)

print("\n" + "="*80)
print("开始绘制交叉验证 ROC 曲线...")
print("="*80)

# ========== 绘制各模型的 CV ROC 曲线 ==========
# 1. SVM
fig, ax = plt.subplots(figsize=(10, 8))
plot_cv_roc_curve(svm, X_train, y_train_series, cv_folds_indices, "SVM", ax=ax)
plt.tight_layout()
plt.savefig("/home/phl/PHL/Car-T/model_v1/output/cv_roc_svm.png", dpi=300)
plt.show()

# 2. XGBoost
fig, ax = plt.subplots(figsize=(10, 8))
plot_cv_roc_curve(xgb_model, X_train, y_train_series, cv_folds_indices, "XGBoost", ax=ax)
plt.tight_layout()
plt.savefig("/home/phl/PHL/Car-T/model_v1/output/cv_roc_xgboost.png", dpi=300)
plt.show()

# 3. LightGBM
fig, ax = plt.subplots(figsize=(10, 8))
plot_cv_roc_curve(lgb_model, X_train, y_train_series, cv_folds_indices, "LightGBM", ax=ax, is_lgb=True)
plt.tight_layout()
plt.savefig("/home/phl/PHL/Car-T/model_v1/output/cv_roc_lightgbm.png", dpi=300)
plt.show()

# 4. Logistic Regression
fig, ax = plt.subplots(figsize=(10, 8))
plot_cv_roc_curve(log_reg, X_train, y_train_series, cv_folds_indices, "Logistic Regression", ax=ax)
plt.tight_layout()
plt.savefig("/home/phl/PHL/Car-T/model_v1/output/cv_roc_logistic.png", dpi=300)
plt.show()

# 5. Random Forest
fig, ax = plt.subplots(figsize=(10, 8))
plot_cv_roc_curve(rf, X_train, y_train_series, cv_folds_indices, "Random Forest", ax=ax)
plt.tight_layout()
plt.savefig("/home/phl/PHL/Car-T/model_v1/output/cv_roc_rf.png", dpi=300)
plt.show()

# 6. MLP
fig, ax = plt.subplots(figsize=(10, 8))
plot_cv_roc_curve(mlp, X_train, y_train_series, cv_folds_indices, "MLP", ax=ax)
plt.tight_layout()
plt.savefig("/home/phl/PHL/Car-T/model_v1/output/cv_roc_mlp.png", dpi=300)
plt.show()

# 7. KNN
fig, ax = plt.subplots(figsize=(10, 8))
plot_cv_roc_curve(knn, X_train, y_train_series, cv_folds_indices, "KNN", ax=ax)
plt.tight_layout()
plt.savefig("/home/phl/PHL/Car-T/model_v1/output/cv_roc_knn.png", dpi=300)
plt.show()

# 8. AdaBoost
fig, ax = plt.subplots(figsize=(10, 8))
plot_cv_roc_curve(ada, X_train, y_train_series, cv_folds_indices, "AdaBoost", ax=ax)
plt.tight_layout()
plt.savefig("/home/phl/PHL/Car-T/model_v1/output/cv_roc_adaboost.png", dpi=300)
plt.show()

# 9. Decision Tree
fig, ax = plt.subplots(figsize=(10, 8))
plot_cv_roc_curve(dt, X_train, y_train_series, cv_folds_indices, "Decision Tree", ax=ax)
plt.tight_layout()
plt.savefig("/home/phl/PHL/Car-T/model_v1/output/cv_roc_decision_tree.png", dpi=300)
plt.show()

# 10. Naive Bayes
fig, ax = plt.subplots(figsize=(10, 8))
plot_cv_roc_curve(nb, X_train, y_train_series, cv_folds_indices, "Naive Bayes", ax=ax)
plt.tight_layout()
plt.savefig("/home/phl/PHL/Car-T/model_v1/output/cv_roc_naive_bayes.png", dpi=300)
plt.show()

print("\n✅ 交叉验证 ROC 曲线绘制完成!")

# 📊 最终模型测试集验证与ROC曲线绘制

## 工作流程说明

经过网格搜索 (`GridSearchCV`) 和交叉验证后，我们已经得到了每个模型的最佳参数。接下来的步骤：

1. **使用最佳参数在全量训练集上重新训练** 
   - GridSearchCV 的 `refit=True` 参数已自动完成此步骤
   - `final_models` 字典中存储的就是在完整训练集上重训后的最佳模型

2. **在测试集上进行预测和验证** 
   - 使用重训后的模型对测试集进行预测
   - 计算测试集上的性能指标（AUC、准确率等）

3. **绘制测试集ROC曲线**
   - 为每个模型绘制在测试集上的ROC曲线
   - 对比不同模型的测试集性能

In [ ]:
# 最终模型测试集验证与ROC曲线绘制
# =============================================================================
# 步骤1: 确认模型已使用最佳参数在全量训练集上重新训练
# =============================================================================
# GridSearchCV 已自动完成这一步 (refit=True)
# final_models 字典中的模型都是最终的、已重训的模型

print("="*80)
print("步骤1: 确认最佳模型 (已在完整训练集上重训)")
print("="*80)
print(f"可用的最佳模型: {list(final_models.keys())}")
print(f"训练集大小: X_train {X_train.shape}, y_train {len(y_train)}")
print(f"测试集大小: X_test {X_test.shape}, y_test {len(y_test)}")
print()

# =============================================================================
# 步骤2: 在测试集上进行预测和评估
# =============================================================================
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

print("="*80)
print("步骤2: 在测试集上进行验证")
print("="*80)

# 存储测试集性能
test_results = []

for model_name, model in final_models.items():
    print(f"\n正在评估 {model_name} ...")
    
    # 预测概率
    if model_name == 'LightGBM':
        # LightGBM 使用特殊的预测方法
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 预测类别 (使用默认阈值 0.5)
    y_pred = (y_pred_proba >= 0.5).astype(int)
    
    # 计算各项指标
    test_auc = roc_auc_score(y_test, y_pred_proba)
    test_acc = accuracy_score(y_test, y_pred)
    test_precision = precision_score(y_test, y_pred, zero_division=0)
    test_recall = recall_score(y_test, y_pred, zero_division=0)
    test_f1 = f1_score(y_test, y_pred, zero_division=0)
    
    # 存储结果
    test_results.append({
        'Model': model_name,
        'Test AUC': test_auc,
        'Test Accuracy': test_acc,
        'Test Precision': test_precision,
        'Test Recall': test_recall,
        'Test F1': test_f1
    })
    
    print(f"  测试集 AUC: {test_auc:.4f}")
    print(f"  测试集准确率: {test_acc:.4f}")
    print(f"  测试集精确率: {test_precision:.4f}")
    print(f"  测试集召回率: {test_recall:.4f}")
    print(f"  测试集 F1 分数: {test_f1:.4f}")

# 转换为 DataFrame 并排序
df_test_results = pd.DataFrame(test_results).sort_values('Test AUC', ascending=False)
print("\n" + "="*80)
print("测试集性能汇总 (按 AUC 排序)")
print("="*80)
print(df_test_results.to_string(index=False))

# 保存测试集结果
output_dir = '/home/phl/PHL/Car-T/model_v1/output'
df_test_results.to_csv(os.path.join(output_dir, 'test_set_results.csv'), index=False)
print(f"\n✅ 测试集结果已保存至: {os.path.join(output_dir, 'test_set_results.csv')}")
print()

In [ ]:
# =============================================================================
# 步骤3: 绘制测试集ROC曲线 - 所有模型在一张图上
# =============================================================================
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

print("="*80)
print("步骤3: 绘制测试集ROC曲线")
print("="*80)

# 定义颜色方案 (使用鲜艳的颜色)
colors = {
    'SVM': '#E74C3C',           # 红色
    'XGBoost': '#3498DB',       # 蓝色
    'LightGBM': '#2ECC71',      # 绿色
    'Logistic Regression': '#F39C12',  # 橙色
    'Random Forest': '#9B59B6', # 紫色
    'MLP': '#1ABC9C',           # 青色
    'KNN': '#E67E22',           # 深橙色
    'AdaBoost': '#34495E',      # 深灰色
    'Decision Tree': '#16A085', # 深青色
    'Naive Bayes': '#C0392B'    # 深红色
}

# 创建图形
plt.figure(figsize=(12, 10))

# 为每个模型绘制ROC曲线
for model_name, model in final_models.items():
    print(f"正在绘制 {model_name} 的测试集ROC曲线...")
    
    # 预测概率
    if model_name == 'LightGBM':
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 计算ROC曲线
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    # 绘制曲线
    color = colors.get(model_name, '#95A5A6')  # 默认灰色
    plt.plot(fpr, tpr, color=color, lw=2.5, 
             label=f'{model_name} (AUC = {roc_auc:.3f})', alpha=0.8)

# 绘制对角线（随机分类器）
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier (AUC = 0.500)', alpha=0.6)

# 图形美化
plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('False Positive Rate (FPR)', fontsize=14, fontweight='bold')
plt.ylabel('True Positive Rate (TPR)', fontsize=14, fontweight='bold')
plt.title('ROC Curves on Test Set - All Models', fontsize=16, fontweight='bold', pad=20)
plt.legend(loc='lower right', fontsize=11, framealpha=0.95)
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()

# 保存图形
output_path = os.path.join(output_dir, 'test_set_roc_curves_all_models.png')
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f"\n✅ 测试集ROC曲线已保存至: {output_path}")

# 同时保存为PDF格式（用于论文发表）
output_path_pdf = os.path.join(output_dir, 'test_set_roc_curves_all_models.pdf')
plt.savefig(output_path_pdf, dpi=300, bbox_inches='tight')
print(f"✅ 测试集ROC曲线(PDF)已保存至: {output_path_pdf}")

plt.show()
print()

In [ ]:
# =============================================================================
# 步骤4: 为每个模型单独绘制测试集ROC曲线（高清大图）
# =============================================================================
print("="*80)
print("步骤4: 为每个模型单独绘制测试集ROC曲线")
print("="*80)

# 创建子目录保存单独的ROC曲线图
individual_roc_dir = os.path.join(output_dir, 'individual_test_roc_curves')
if not os.path.exists(individual_roc_dir):
    os.makedirs(individual_roc_dir)
    print(f"创建目录: {individual_roc_dir}")

# 为每个模型单独绘制ROC曲线
for model_name, model in final_models.items():
    print(f"\n正在为 {model_name} 绘制单独的测试集ROC曲线...")
    
    # 预测概率
    if model_name == 'LightGBM':
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 计算ROC曲线
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    # 创建新图形
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # 绘制ROC曲线
    color = colors.get(model_name, '#3498DB')
    ax.plot(fpr, tpr, color=color, lw=3, 
            label=f'{model_name}\n(AUC = {roc_auc:.4f})', alpha=0.9)
    
    # 绘制对角线
    ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier (AUC = 0.500)', alpha=0.6)
    
    # 填充ROC曲线下的面积
    ax.fill_between(fpr, tpr, alpha=0.2, color=color)
    
    # 图形美化
    ax.set_xlim([-0.02, 1.02])
    ax.set_ylim([-0.02, 1.02])
    ax.set_xlabel('False Positive Rate (FPR)', fontsize=14, fontweight='bold')
    ax.set_ylabel('True Positive Rate (TPR)', fontsize=14, fontweight='bold')
    ax.set_title(f'ROC Curve - {model_name}\nTest Set Performance', 
                 fontsize=16, fontweight='bold', pad=20)
    ax.legend(loc='lower right', fontsize=12, framealpha=0.95)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    # 添加统计信息文本框
    test_acc = accuracy_score(y_test, (y_pred_proba >= 0.5).astype(int))
    textstr = f'Test Samples: {len(y_test)}\nAccuracy: {test_acc:.4f}\nAUC: {roc_auc:.4f}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    ax.text(0.98, 0.02, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment='bottom', horizontalalignment='right', bbox=props)
    
    plt.tight_layout()
    
    # 保存图形
    safe_name = model_name.replace(' ', '_').lower()
    output_path = os.path.join(individual_roc_dir, f'test_roc_{safe_name}.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    
    # 同时保存PDF格式
    output_path_pdf = os.path.join(individual_roc_dir, f'test_roc_{safe_name}.pdf')
    plt.savefig(output_path_pdf, dpi=300, bbox_inches='tight')
    
    plt.close()
    print(f"  ✅ 已保存: {output_path}")

print(f"\n✅ 所有单独的测试集ROC曲线已保存至: {individual_roc_dir}")
print()

In [ ]:
# =============================================================================
# 步骤5: 对比训练集和测试集性能（检查过拟合）
# =============================================================================
print("="*80)
print("步骤5: 训练集 vs 测试集性能对比")
print("="*80)

# 计算训练集性能
train_results = []

for model_name, model in final_models.items():
    # 预测训练集
    if model_name == 'LightGBM':
        y_train_pred_proba = model.predict_proba(X_train)[:, 1]
    else:
        y_train_pred_proba = model.predict_proba(X_train)[:, 1]
    
    train_auc = roc_auc_score(y_train, y_train_pred_proba)
    
    train_results.append({
        'Model': model_name,
        'Train AUC': train_auc
    })

df_train_results = pd.DataFrame(train_results)

# 合并训练集和测试集结果
df_comparison = df_test_results[['Model', 'Test AUC']].merge(
    df_train_results, on='Model'
)

# 计算过拟合程度 (训练集AUC - 测试集AUC)
df_comparison['Overfit Gap'] = df_comparison['Train AUC'] - df_comparison['Test AUC']
df_comparison = df_comparison.sort_values('Test AUC', ascending=False)

print(df_comparison.to_string(index=False))
print()

# 可视化对比
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 左图: 训练集 vs 测试集 AUC对比
models_list = df_comparison['Model'].tolist()
train_aucs = df_comparison['Train AUC'].tolist()
test_aucs = df_comparison['Test AUC'].tolist()

x = np.arange(len(models_list))
width = 0.35

bars1 = ax1.bar(x - width/2, train_aucs, width, label='Train AUC', 
                alpha=0.8, color='#3498DB')
bars2 = ax1.bar(x + width/2, test_aucs, width, label='Test AUC', 
                alpha=0.8, color='#E74C3C')

ax1.set_xlabel('Model', fontsize=12, fontweight='bold')
ax1.set_ylabel('AUC Score', fontsize=12, fontweight='bold')
ax1.set_title('Train vs Test AUC Comparison', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(models_list, rotation=45, ha='right')
ax1.legend(fontsize=11)
ax1.set_ylim([0.5, 1.0])
ax1.grid(True, alpha=0.3, axis='y')

# 右图: 过拟合程度
overfit_gaps = df_comparison['Overfit Gap'].tolist()
colors_overfit = ['#27AE60' if gap < 0.1 else '#F39C12' if gap < 0.2 else '#E74C3C' 
                  for gap in overfit_gaps]

bars3 = ax2.bar(x, overfit_gaps, alpha=0.8, color=colors_overfit)

ax2.set_xlabel('Model', fontsize=12, fontweight='bold')
ax2.set_ylabel('Overfit Gap (Train AUC - Test AUC)', fontsize=12, fontweight='bold')
ax2.set_title('Overfitting Analysis', fontsize=14, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(models_list, rotation=45, ha='right')
ax2.axhline(y=0.1, color='orange', linestyle='--', linewidth=2, alpha=0.7, 
            label='Moderate Overfit (0.1)')
ax2.axhline(y=0.2, color='red', linestyle='--', linewidth=2, alpha=0.7, 
            label='High Overfit (0.2)')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()

# 保存图形
output_path = os.path.join(output_dir, 'train_vs_test_comparison.png')
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f"✅ 训练集vs测试集对比图已保存至: {output_path}")

output_path_pdf = os.path.join(output_dir, 'train_vs_test_comparison.pdf')
plt.savefig(output_path_pdf, dpi=300, bbox_inches='tight')
print(f"✅ 训练集vs测试集对比图(PDF)已保存至: {output_path_pdf}")

plt.show()

# 保存对比结果
df_comparison.to_csv(os.path.join(output_dir, 'train_test_comparison.csv'), index=False)
print(f"✅ 对比结果已保存至: {os.path.join(output_dir, 'train_test_comparison.csv')}")
print()

In [ ]:
# =============================================================================
# 步骤6: 绘制Top3模型的测试集ROC曲线对比图（用于论文展示）
# =============================================================================
print("="*80)
print("步骤6: 绘制Top3模型的测试集ROC曲线对比")
print("="*80)

# 选择测试集AUC最高的前3个模型
top3_models = df_test_results.head(3)['Model'].tolist()
print(f"Top 3 模型: {top3_models}")

# 创建高质量图形
fig, ax = plt.subplots(figsize=(10, 8))

# 定义更鲜艳的颜色
top_colors = ['#E74C3C', '#3498DB', '#2ECC71']  # 红、蓝、绿

for idx, model_name in enumerate(top3_models):
    model = final_models[model_name]
    
    # 预测概率
    if model_name == 'LightGBM':
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 计算ROC曲线
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    # 绘制ROC曲线
    ax.plot(fpr, tpr, color=top_colors[idx], lw=3.5, 
            label=f'{model_name} (AUC = {roc_auc:.4f})', alpha=0.9)
    
    print(f"  {idx+1}. {model_name}: AUC = {roc_auc:.4f}")

# 绘制对角线
ax.plot([0, 1], [0, 1], 'k--', lw=2.5, label='Random Classifier', alpha=0.6)

# 图形美化
ax.set_xlim([-0.02, 1.02])
ax.set_ylim([-0.02, 1.02])
ax.set_xlabel('False Positive Rate (FPR)', fontsize=16, fontweight='bold')
ax.set_ylabel('True Positive Rate (TPR)', fontsize=16, fontweight='bold')
ax.set_title('ROC Curves - Top 3 Models (Test Set)', 
             fontsize=18, fontweight='bold', pad=20)
ax.legend(loc='lower right', fontsize=14, framealpha=0.95, edgecolor='black', fancybox=True)
ax.grid(True, alpha=0.3, linestyle='--', linewidth=1)

# 设置刻度字体大小
ax.tick_params(axis='both', which='major', labelsize=12)

plt.tight_layout()

# 保存高分辨率图形（用于论文）
output_path = os.path.join(output_dir, 'test_roc_top3_models_publication.png')
plt.savefig(output_path, dpi=600, bbox_inches='tight', facecolor='white')
print(f"\n✅ Top3模型ROC曲线(高清PNG)已保存至: {output_path}")

output_path_pdf = os.path.join(output_dir, 'test_roc_top3_models_publication.pdf')
plt.savefig(output_path_pdf, dpi=600, bbox_inches='tight')
print(f"✅ Top3模型ROC曲线(PDF)已保存至: {output_path_pdf}")

plt.show()
print()

## 📋 总结

### 完整工作流程回顾

1. **网格搜索与交叉验证** ✅ (单元格23)
   - 使用 `GridSearchCV` 对每个模型进行超参数调优
   - 在预定义的5折交叉验证上搜索最佳参数
   - 自动使用最佳参数在完整训练集上重新训练（`refit=True`）

2. **测试集验证** ✅ (刚刚完成)
   - 使用重训后的最佳模型对测试集进行预测
   - 计算测试集上的多项性能指标（AUC、准确率、精确率、召回率、F1分数）
   - 保存测试集结果到CSV文件

3. **ROC曲线绘制** ✅ (刚刚完成)
   - 绘制所有模型在测试集上的ROC曲线（一张图）
   - 为每个模型单独绘制高清ROC曲线
   - 绘制Top3模型的对比ROC曲线（用于论文发表）

4. **过拟合分析** ✅ (刚刚完成)
   - 对比训练集和测试集性能
   - 计算过拟合程度（Train AUC - Test AUC）
   - 可视化过拟合分析结果

### 输出文件清单

所有文件保存在 `/home/phl/PHL/Car-T/model_v1/output/` 目录下：

- `test_set_results.csv` - 测试集性能汇总表
- `train_test_comparison.csv` - 训练集vs测试集对比
- `test_set_roc_curves_all_models.png/pdf` - 所有模型ROC曲线
- `train_vs_test_comparison.png/pdf` - 训练集vs测试集对比图
- `test_roc_top3_models_publication.png/pdf` - Top3模型ROC曲线（高清）
- `individual_test_roc_curves/` - 每个模型的单独ROC曲线

### 关键要点

✅ **无需手动重训练**: `GridSearchCV` 的 `refit=True` 已自动完成

✅ **使用相同的数据划分**: 测试集来自初始划分，确保公平比较

✅ **多格式输出**: 同时生成PNG和PDF格式，满足不同需求

✅ **过拟合检测**: 通过对比训练集和测试集性能，识别过拟合问题

In [ ]:
# =============================================================================
# 额外: 查看最佳模型的详细信息和混淆矩阵
# =============================================================================
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

print("="*80)
print("最佳模型详细分析")
print("="*80)

# 获取测试集AUC最高的模型
best_model_name = df_test_results.iloc[0]['Model']
best_model = final_models[best_model_name]
best_test_auc = df_test_results.iloc[0]['Test AUC']

print(f"\n🏆 最佳模型: {best_model_name}")
print(f"   测试集 AUC: {best_test_auc:.4f}")
print()

# 预测
if best_model_name == 'LightGBM':
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]
else:
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]

y_pred = (y_pred_proba >= 0.5).astype(int)

# 打印详细分类报告
print("分类报告:")
print(classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1']))

# 绘制混淆矩阵
fig, ax = plt.subplots(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Class 0', 'Class 1'])
disp.plot(cmap='Blues', ax=ax, values_format='d')
ax.set_title(f'Confusion Matrix - {best_model_name}\n(Test Set)', 
             fontsize=14, fontweight='bold', pad=15)

plt.tight_layout()

# 保存混淆矩阵
output_path = os.path.join(output_dir, f'confusion_matrix_{best_model_name.replace(" ", "_").lower()}.png')
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f"\n✅ 混淆矩阵已保存至: {output_path}")

plt.show()

# 查看模型的最佳参数
print(f"\n{best_model_name} 的最佳参数:")
if hasattr(best_model, 'named_steps'):
    # 如果是Pipeline，获取classifier部分的参数
    classifier = best_model.named_steps['classifier']
    params = classifier.get_params()
    # 只显示非默认的关键参数
    key_params = {k: v for k, v in params.items() 
                  if not k.endswith('_') and not callable(v) and v is not None}
    for param, value in sorted(key_params.items())[:10]:  # 只显示前10个参数
        print(f"  {param}: {value}")
else:
    print("  ", best_model.get_params())

print(f"\n{'='*80}")
print("分析完成！")
print(f"{'='*80}")

In [ ]:
# 模型自带的特征重要性排序（支持 RF、XGBoost、LightGBM）
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

# =============================================================================
# 1. 定义通用的特征名称提取函数
# =============================================================================
def get_feature_names_from_preprocessor(preprocessor):
    """
    从 ColumnTransformer 中提取所有转换后的特征名称
    兼容 sklearn >= 1.0 的 get_feature_names_out() 方法
    """
    # 优先使用 sklearn 内置方法
    try:
        return list(preprocessor.get_feature_names_out())
    except AttributeError:
        pass
    
    # 备用方案：手动遍历
    feature_names = []
    for name, transformer, columns in preprocessor.transformers_:
        if name == 'num':
            feature_names.extend(columns)
        elif name == 'cat':
            ohe = transformer.named_steps['onehot']
            ohe_feature_names = ohe.get_feature_names_out(columns)
            feature_names.extend(ohe_feature_names)
        elif name == 'remainder' and transformer == 'passthrough':
            if isinstance(columns, list):
                feature_names.extend(columns)
    return feature_names


def extract_feature_importance(pipeline, model_name):
    """
    从 Pipeline 中提取特征重要性
    
    参数:
        pipeline: 包含 preprocessor 和 classifier 的 Pipeline 对象
        model_name: 模型名称 (用于日志输出)
    
    返回:
        tuple: (importances, feature_names)
    """
    # 获取分类器和预处理器
    classifier = pipeline.named_steps['classifier']
    preprocessor = pipeline.named_steps['preprocessor']
    
    # 获取特征重要性
    importances = classifier.feature_importances_
    
    # 获取特征名称
    feature_names = get_feature_names_from_preprocessor(preprocessor)
    
    # 验证数量匹配
    if len(feature_names) != len(importances):
        print(f" {model_name}: 特征数量不匹配 ({len(feature_names)} vs {len(importances)})")
        feature_names = [f"feature_{i}" for i in range(len(importances))]
    
    return importances, feature_names


def plot_feature_importance(importances, feature_names, model_name, top_n=20, 
                            output_dir="/home/phl/PHL/Car-T/model_v1/output",
                            importance_type="default"):
    """
    绘制特征重要性条形图
    
    参数:
        importances: 特征重要性数组
        feature_names: 特征名称列表
        model_name: 模型名称
        top_n: 显示前 N 个特征
        output_dir: 输出目录
        importance_type: 重要性类型 (用于标题显示)
    """
    # 创建 DataFrame 并排序
    feature_imp_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values(by='importance', ascending=False).reset_index(drop=True)
    
    # 取 Top N
    top_features = feature_imp_df.head(top_n)
    plot_data = top_features.sort_values(by='importance', ascending=True)
    
    # 绘图
    fig, ax = plt.subplots(figsize=(10, 8))
    y_ticks = np.arange(len(plot_data))
    
    # 根据模型类型设置颜色
    color_map = {
        'Random Forest': '#1f77b4',
        'XGBoost': '#ff7f0e',
        'LightGBM': '#2ca02c'
    }
    color = color_map.get(model_name, '#1f77b4')
    
    ax.barh(y_ticks, plot_data['importance'], color=color)
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(plot_data['feature'])
    
    # 设置标题 (包含重要性类型说明)
    title_suffix = {
        'Random Forest': 'MDI (Mean Decrease Impurity)',
        'XGBoost': 'Gain',
        'LightGBM': 'Split'
    }
    ax.set_title(f"{model_name} Feature Importances (Top {top_n})\n[{title_suffix.get(model_name, '')}]", 
                 fontsize=14)
    ax.set_xlabel("Importance Score", fontsize=12)
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    
    # 保存
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    filename = f"{model_name.lower().replace(' ', '_')}_feature_importance.png"
    output_path = os.path.join(output_dir, filename)
    plt.savefig(output_path, bbox_inches='tight')
    print(f" {model_name} 特征重要性图已保存至: {output_path}")
    plt.show()
    
    return feature_imp_df


# =============================================================================
# 2. 绘制各模型的特征重要性
# =============================================================================
print("=" * 80)
print("开始提取和绘制各模型的特征重要性...")
print("=" * 80)

# 存储所有模型的特征重要性结果
all_importance_results = {}

# --- Random Forest ---
print("\n Random Forest:")
rf_importances, rf_feature_names = extract_feature_importance(rf, "Random Forest")
rf_imp_df = plot_feature_importance(rf_importances, rf_feature_names, "Random Forest")
all_importance_results['Random Forest'] = rf_imp_df

# # --- XGBoost ---
# print("\n XGBoost:")
# xgb_importances, xgb_feature_names = extract_feature_importance(xgb_model, "XGBoost")
# xgb_imp_df = plot_feature_importance(xgb_importances, xgb_feature_names, "XGBoost")
# all_importance_results['XGBoost'] = xgb_imp_df

# --- LightGBM ---
print("\n LightGBM:")
lgb_importances, lgb_feature_names = extract_feature_importance(lgb_model, "LightGBM")
lgb_imp_df = plot_feature_importance(lgb_importances, lgb_feature_names, "LightGBM")
all_importance_results['LightGBM'] = lgb_imp_df

# =============================================================================
# 3. 多模型特征重要性对比图
# =============================================================================
print("\n" + "=" * 80)
print("绘制多模型特征重要性对比图...")
print("=" * 80)

# 合并所有模型的 Top 10 特征
top_n_compare = 10
comparison_data = []

for model_name, imp_df in all_importance_results.items():
    top_features = imp_df.head(top_n_compare).copy()
    top_features['model'] = model_name
    comparison_data.append(top_features)

comparison_df = pd.concat(comparison_data, ignore_index=True)

# 找出所有模型中出现过的 Top 特征的并集
all_top_features = comparison_df['feature'].unique()

# 创建对比热力图数据
heatmap_data = pd.DataFrame(index=all_top_features)
for model_name, imp_df in all_importance_results.items():
    imp_dict = dict(zip(imp_df['feature'], imp_df['importance']))
    heatmap_data[model_name] = [imp_dict.get(f, 0) for f in all_top_features]

# 按平均重要性排序
heatmap_data['mean'] = heatmap_data.mean(axis=1)
heatmap_data = heatmap_data.sort_values('mean', ascending=False).drop(columns='mean').head(15)

# 绘制热力图
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(heatmap_data.values, cmap='YlOrRd', aspect='auto')

# 设置坐标轴
ax.set_xticks(np.arange(len(heatmap_data.columns)))
ax.set_yticks(np.arange(len(heatmap_data.index)))
ax.set_xticklabels(heatmap_data.columns)
ax.set_yticklabels(heatmap_data.index)

# 添加颜色条
cbar = ax.figure.colorbar(im, ax=ax)
cbar.ax.set_ylabel("Importance Score", rotation=-90, va="bottom")

# 在单元格中添加数值
for i in range(len(heatmap_data.index)):
    for j in range(len(heatmap_data.columns)):
        text = ax.text(j, i, f"{heatmap_data.iloc[i, j]:.3f}",
                       ha="center", va="center", color="black", fontsize=9)

ax.set_title("Feature Importance Comparison Across Models", fontsize=14)
plt.tight_layout()

output_path = os.path.join("/home/phl/PHL/Car-T/model_v1/output", "feature_importance_comparison.png")
plt.savefig(output_path, bbox_inches='tight')
print(f"✅ 多模型对比图已保存至: {output_path}")
plt.show()

# =============================================================================
# 4. 打印各模型 Top 10 特征汇总表
# =============================================================================
print("\n" + "=" * 80)
print("各模型 Top 10 特征汇总")
print("=" * 80)

for model_name, imp_df in all_importance_results.items():
    print(f"\n【{model_name}】")
    print(imp_df.head(10).to_string(index=False))

主要特性:可以通过修改 models_to_analyze 列表来选择要分析的模型。
```
特性	            说明
多模型支持	        通过 models_to_analyze 列表灵活选择要分析的模型
自动选择Explainer	树模型用 TreeExplainer，线性模型用 LinearExplainer，其他用 KernelExplainer
批量生成图表	    每个模型生成 3 种图：Bar Plot、Beeswarm Plot、自定义条形图
多模型对比热力图	 自动生成特征重要性对比热力图
结果保存	       所有图片自动保存到 output 目录
```

In [ ]:
# 多模型 SHAP 可解释性分析
import matplotlib.cm as cm
import shap

# =============================================================================
# 1. 定义支持 SHAP 分析的模型列表
# =============================================================================
# 不同模型需要不同的 Explainer
tree_based_models = ['LightGBM', 'XGBoost', 'Random Forest', 'Decision Tree', 'AdaBoost']
linear_models = ['Logistic Regression']
kernel_models = ['SVM', 'KNN', 'MLP', 'Naive Bayes']  # 需要 KernelExplainer (较慢)

# 选择要分析的模型 (可根据需要修改)
models_to_analyze = ['SVM', 'Random Forest', 'MLP', 'Logistic Regression']
# models_to_analyze = list(final_models.keys())  # 分析所有模型

# =============================================================================
# 2. 数据预处理 - 获取转换后的特征
# =============================================================================
print("\n" + "=" * 80)
print("准备 SHAP 分析数据...")
print("=" * 80)

# 使用任意一个 Pipeline 获取预处理器 (所有模型共享相同的预处理器)
sample_pipeline = final_models['LightGBM']
preprocessor = sample_pipeline.named_steps['preprocessor']
final_imputer = sample_pipeline.named_steps['final_imputer']

# 对训练数据进行预处理转换
X_train_transformed = preprocessor.transform(X_train)
X_train_transformed = final_imputer.transform(X_train_transformed)

# 获取转换后的特征名称
try:
    feature_names_transformed = preprocessor.get_feature_names_out().tolist()
except AttributeError:
    feature_names_transformed = [f"feature_{i}" for i in range(X_train_transformed.shape[1])]

# 转换为 DataFrame
X_train_for_shap = pd.DataFrame(X_train_transformed, columns=feature_names_transformed)
print(f"原始特征数: {X_train.shape[1]}, 转换后特征数: {X_train_for_shap.shape[1]}")

# 对于 KernelExplainer，使用采样数据加速
X_background = shap.sample(X_train_for_shap, 50)  # 背景数据采样

# =============================================================================
# 3. 定义 SHAP 分析函数
# =============================================================================
def analyze_shap_for_model(model_name, pipeline, X_data, feature_names, output_dir):
    """
    对单个模型进行 SHAP 分析
    
    参数:
        model_name: 模型名称
        pipeline: 训练好的 Pipeline
        X_data: 预处理后的特征数据 (DataFrame)
        feature_names: 特征名称列表
        output_dir: 输出目录
    
    返回:
        dict: 包含 SHAP 值和重要性的字典
    """
    print(f"\n{'='*60}")
    print(f"正在分析: {model_name}")
    print(f"{'='*60}")
    
    classifier = pipeline.named_steps['classifier']
    
    # 根据模型类型选择合适的 Explainer
    try:
        if model_name in ['LightGBM', 'XGBoost', 'Random Forest', 'Decision Tree', 'AdaBoost']:
            # 树模型使用 TreeExplainer (最快)
            explainer = shap.TreeExplainer(classifier)
            shap_values = explainer(X_data)
            
        elif model_name == 'Logistic Regression':
            # 线性模型使用 LinearExplainer
            explainer = shap.LinearExplainer(classifier, X_data)
            shap_values = explainer(X_data)
            
        else:
            # 其他模型使用 KernelExplainer (较慢，使用采样)
            print(f"  使用 KernelExplainer (较慢，采样 100 个样本)...")
            
            def predict_proba_func(x):
                # 创建临时 DataFrame 保持列名
                x_df = pd.DataFrame(x, columns=feature_names)
                # 注意：这里需要绕过 preprocessor，因为数据已经预处理过
                return classifier.predict_proba(x_df)[:, 1]
            
            explainer = shap.KernelExplainer(predict_proba_func, X_background)
            # 仅对部分样本计算 SHAP 值以加速
            X_sample = X_data.sample(n=min(100, len(X_data)), random_state=42)
            shap_values = explainer(X_sample)
            X_data = X_sample  # 更新用于绘图的数据
        
        # 提取 SHAP 值数组
        shap_values_array = np.array(shap_values.values)
        
        # 计算 Mean |SHAP|
        mean_abs_shap = np.mean(np.abs(shap_values_array), axis=0)
        
        # 创建特征重要性 DataFrame
        df_importance = pd.DataFrame({
            'Feature': feature_names,
            'Mean_Abs_SHAP': mean_abs_shap
        }).sort_values(by='Mean_Abs_SHAP', ascending=False).reset_index(drop=True)
        
        # 每个模型生成 3 种图：Bar Plot、Beeswarm Plot、自定义条形图
        # ============ 绘图 1: SHAP Bar Plot ============
        plt.figure(figsize=(12, 8))
        shap.plots.bar(shap_values, max_display=20, show=False)
        plt.title(f'{model_name} - SHAP Feature Importance', fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'shap_bar_{model_name.lower().replace(" ", "_")}.png'), 
                    dpi=300, bbox_inches='tight')
        plt.show()
        
        # ============ 绘图 2: SHAP Summary Plot (Beeswarm) ============
        plt.figure(figsize=(12, 8))
        shap.plots.beeswarm(shap_values, max_display=20, show=False)
        plt.title(f'{model_name} - SHAP Summary Plot', fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'shap_beeswarm_{model_name.lower().replace(" ", "_")}.png'), 
                    dpi=300, bbox_inches='tight')
        plt.show()
        
        # ============ 绘图 3: 自定义 Mean |SHAP| 条形图 ============
        df_plot = df_importance.head(20).sort_values(by='Mean_Abs_SHAP', ascending=True)
        
        plt.figure(figsize=(12, 8))
        colors = cm.RdPu(np.linspace(1, 0.4, len(df_plot)))
        bars = plt.barh(df_plot['Feature'], df_plot['Mean_Abs_SHAP'], color=colors)
        
        for bar in bars:
            plt.text(bar.get_width() + 0.002, bar.get_y() + bar.get_height()/2, 
                     f'{bar.get_width():.4f}', va='center', ha='left', fontsize=8)
        
        plt.xlabel('Mean |SHAP Value|')
        plt.title(f'{model_name} - Feature Importance (Top 20)')
        ax = plt.gca()
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'shap_custom_{model_name.lower().replace(" ", "_")}.png'), 
                    dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f" {model_name} SHAP 分析完成!")
        print(f"\nTop 10 特征:")
        print(df_importance.head(10).to_string(index=False))
        
        return {
            'model_name': model_name,
            'shap_values': shap_values,
            'importance_df': df_importance,
            'mean_abs_shap': mean_abs_shap
        }
        
    except Exception as e:
        print(f" {model_name} SHAP 分析失败: {e}")
        return None

# =============================================================================
# 4. 批量执行 SHAP 分析
# =============================================================================
print("\n" + "=" * 80)
print("开始多模型 SHAP 可解释性分析")
print("=" * 80)

output_dir = '/home/phl/PHL/Car-T/model_v1/output/shap_analysis'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 存储所有模型的结果
all_shap_results = {}

for model_name in models_to_analyze:
    if model_name in final_models:
        result = analyze_shap_for_model(
            model_name=model_name,
            pipeline=final_models[model_name],
            X_data=X_train_for_shap.copy(),
            feature_names=feature_names_transformed,
            output_dir=output_dir
        )
        if result is not None:
            all_shap_results[model_name] = result
    else:
        print(f" 模型 {model_name} 未找到，跳过...")

# =============================================================================
# 6. 多模型 SHAP 对比分析
# =============================================================================
if len(all_shap_results) > 1:
    print("\n" + "=" * 80)
    print("多模型 SHAP 特征重要性对比")
    print("=" * 80)
    
    # 创建对比 DataFrame
    comparison_data = {}
    for model_name, result in all_shap_results.items():
        importance_dict = dict(zip(
            result['importance_df']['Feature'], 
            result['importance_df']['Mean_Abs_SHAP']
        ))
        comparison_data[model_name] = importance_dict
    
    # 获取所有模型 Top 15 特征的并集
    all_top_features = set()
    for model_name, result in all_shap_results.items():
        top_features = result['importance_df'].head(15)['Feature'].tolist()
        all_top_features.update(top_features)
    
    # 构建对比矩阵
    comparison_df = pd.DataFrame(index=list(all_top_features))
    for model_name, importance_dict in comparison_data.items():
        comparison_df[model_name] = [importance_dict.get(f, 0) for f in comparison_df.index]
    
    # 按平均值排序
    comparison_df['Mean'] = comparison_df.mean(axis=1)
    comparison_df = comparison_df.sort_values('Mean', ascending=False).drop(columns='Mean')
    
    # 绘制特征重要性对比热力图
    plt.figure(figsize=(12, 10))
    im = plt.imshow(comparison_df.values, cmap='YlOrRd', aspect='auto')
    
    plt.xticks(np.arange(len(comparison_df.columns)), comparison_df.columns, rotation=45, ha='right')
    plt.yticks(np.arange(len(comparison_df.index)), comparison_df.index)
    
    # 添加数值标签
    for i in range(len(comparison_df.index)):
        for j in range(len(comparison_df.columns)):
            text = plt.text(j, i, f'{comparison_df.iloc[i, j]:.3f}',
                           ha='center', va='center', color='black', fontsize=8)
    
    plt.colorbar(im, label='Mean |SHAP Value|')
    plt.title('Multi-Model SHAP Feature Importance Comparison', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'shap_multi_model_comparison.png'), dpi=300, bbox_inches='tight')
    plt.show()
    
    # 打印对比表
    print("\n特征重要性对比表:")
    print(comparison_df.round(4).to_string())

print("\n" + "=" * 80)
print(" 多模型 SHAP 分析全部完成!")
print("=" * 80)

包含的评估指标
```
指标                说明	        计算公式
Accuracy	  准确率	        (TP+TN) / (TP+TN+FP+FN)
AUC	          ROC曲线下面积	    基于预测概率计算
Sensitivity	  灵敏度/真阳性率	 TP / (TP+FN)
Specificity	  特异性/真阴性率	 TN / (TN+FP)
Precision	  精确率	        TP / (TP+FP)
Recall	          召回率	        TP / (TP+FN)
F1 Score	  F1分数	        2×(Precision×Recall)/(Precision+Recall)
G-Mean	          几何均值	   √(Sensitivity × Specificity)
```
输出内容
```
训练集评估结果表 - 所有模型在训练集上的指标
测试集评估结果表 - 所有模型在测试集上的指标
各模型详细对比 - 训练集与测试集的差异（用于检测过拟合）
CSV 文件保存 - 便于后续分析
可视化对比图 - 4个子图展示不同指标对比
```

In [ ]:
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score, recall_score, 
    f1_score, confusion_matrix, classification_report
)

# =============================================================================
# 1. 定义计算所有评估指标的函数
# =============================================================================
def calculate_metrics(y_true, y_pred, y_proba=None):
    """
    计算完整的分类评估指标
    
    参数:
        y_true: 真实标签
        y_pred: 预测标签
        y_proba: 预测概率 (用于计算 AUC)
    
    返回:
        dict: 包含所有指标的字典
    """
    # 混淆矩阵: [[TN, FP], [FN, TP]]
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # 基础指标
    accuracy = accuracy_score(y_true, y_pred)
    
    # 灵敏度 (Sensitivity) = 召回率 (Recall) = TP / (TP + FN)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    recall = sensitivity  # 召回率与灵敏度相同
    
    # 特异性 (Specificity) = TN / (TN + FP)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # 精确率 (Precision) = TP / (TP + FP)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    # F1 分数 = 2 * (Precision * Recall) / (Precision + Recall)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    # G均值 (G-Mean) = sqrt(Sensitivity * Specificity)
    g_mean = np.sqrt(sensitivity * specificity)
    
    # AUC (如果有概率预测)
    auc = roc_auc_score(y_true, y_proba) if y_proba is not None else None
    
    return {
        'Accuracy': accuracy,
        'AUC': auc,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'G-Mean': g_mean,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    }

# =============================================================================
# 2. 计算训练集和测试集的评估指标
# =============================================================================
print("=" * 100)
print("多模型分类评估指标汇总")
print("=" * 100)

# 存储所有结果
train_results = []
test_results = []

for model_name, model in final_models.items():
    # ===================== 训练集评估 =====================
    y_train_pred = model.predict(X_train)
    y_train_proba = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else None
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_proba)
    train_metrics['Model'] = model_name
    train_results.append(train_metrics)
    
    # ===================== 测试集评估 =====================
    y_test_pred = model.predict(X_test)
    y_test_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)
    test_metrics['Model'] = model_name
    test_results.append(test_metrics)

# =============================================================================
# 3. 显示训练集评估结果
# =============================================================================
print("\n" + "=" * 100)
print("【训练集评估结果】")
print("=" * 100)

df_train_results = pd.DataFrame(train_results)
# 调整列顺序
cols_order = ['Model', 'Accuracy', 'AUC', 'Sensitivity', 'Specificity', 
              'Precision', 'Recall', 'F1 Score', 'G-Mean', 'TP', 'TN', 'FP', 'FN']
df_train_results = df_train_results[cols_order]

# 按 AUC 降序排列
df_train_results = df_train_results.sort_values('AUC', ascending=False).reset_index(drop=True)

# 格式化显示
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print(df_train_results.to_string(index=False))

# =============================================================================
# 4. 显示测试集评估结果
# =============================================================================
print("\n" + "=" * 100)
print("【测试集评估结果】")
print("=" * 100)

df_test_results = pd.DataFrame(test_results)
df_test_results = df_test_results[cols_order]
df_test_results = df_test_results.sort_values('AUC', ascending=False).reset_index(drop=True)

print(df_test_results.to_string(index=False))

# =============================================================================
# 5. 详细输出每个模型的指标
# =============================================================================
print("\n" + "=" * 100)
print("【各模型详细评估指标】")
print("=" * 100)

for model_name in final_models.keys():
    train_row = df_train_results[df_train_results['Model'] == model_name].iloc[0]
    test_row = df_test_results[df_test_results['Model'] == model_name].iloc[0]
    
    print(f"\n{'─' * 80}")
    print(f"模型: {model_name}")
    print(f"{'─' * 80}")
    print(f"{'指标':<15} {'训练集':>12} {'测试集':>12} {'差异':>12}")
    print(f"{'─' * 51}")
    
    metrics_to_show = ['Accuracy', 'AUC', 'Sensitivity', 'Specificity', 
                       'Precision', 'Recall', 'F1 Score', 'G-Mean']
    
    for metric in metrics_to_show:
        train_val = train_row[metric]
        test_val = test_row[metric]
        diff = train_val - test_val if train_val is not None and test_val is not None else None
        
        train_str = f"{train_val:.4f}" if train_val is not None else "N/A"
        test_str = f"{test_val:.4f}" if test_val is not None else "N/A"
        diff_str = f"{diff:+.4f}" if diff is not None else "N/A"
        
        print(f"{metric:<15} {train_str:>12} {test_str:>12} {diff_str:>12}")
    
    # 混淆矩阵
    print(f"\n混淆矩阵 (测试集):")
    print(f"  TP={int(test_row['TP'])}, TN={int(test_row['TN'])}, FP={int(test_row['FP'])}, FN={int(test_row['FN'])}")

# =============================================================================
# 6. 保存结果到 CSV
# =============================================================================
output_dir = '/home/phl/PHL/Car-T/model_v1/output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df_train_results.to_csv(os.path.join(output_dir, 'train_evaluation_metrics.csv'), index=False)
df_test_results.to_csv(os.path.join(output_dir, 'test_evaluation_metrics.csv'), index=False)

print("\n" + "=" * 100)
print(f"✅ 评估结果已保存至:")
print(f"   - 训练集: {os.path.join(output_dir, 'train_evaluation_metrics.csv')}")
print(f"   - 测试集: {os.path.join(output_dir, 'test_evaluation_metrics.csv')}")
print("=" * 100)

# =============================================================================
# 7. 可视化：指标对比图
# =============================================================================
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 准备数据
models = df_test_results['Model'].tolist()
x = np.arange(len(models))
width = 0.35

# 图1: Accuracy & AUC 对比
ax1 = axes[0, 0]
train_acc = df_train_results.set_index('Model').loc[models, 'Accuracy'].values
test_acc = df_test_results.set_index('Model').loc[models, 'Accuracy'].values
ax1.bar(x - width/2, train_acc, width, label='Train', alpha=0.8)
ax1.bar(x + width/2, test_acc, width, label='Test', alpha=0.8)
ax1.set_ylabel('Score')
ax1.set_title('Accuracy: Train vs Test')
ax1.set_xticks(x)
ax1.set_xticklabels(models, rotation=45, ha='right')
ax1.legend()
ax1.set_ylim([0, 1.1])
ax1.grid(axis='y', alpha=0.3)

# 图2: Sensitivity & Specificity
ax2 = axes[0, 1]
test_sens = df_test_results.set_index('Model').loc[models, 'Sensitivity'].values
test_spec = df_test_results.set_index('Model').loc[models, 'Specificity'].values
ax2.bar(x - width/2, test_sens, width, label='Sensitivity', color='#2ca02c', alpha=0.8)
ax2.bar(x + width/2, test_spec, width, label='Specificity', color='#d62728', alpha=0.8)
ax2.set_ylabel('Score')
ax2.set_title('Sensitivity vs Specificity (Test Set)')
ax2.set_xticks(x)
ax2.set_xticklabels(models, rotation=45, ha='right')
ax2.legend()
ax2.set_ylim([0, 1.1])
ax2.grid(axis='y', alpha=0.3)

# 图3: Precision & Recall
ax3 = axes[1, 0]
test_prec = df_test_results.set_index('Model').loc[models, 'Precision'].values
test_rec = df_test_results.set_index('Model').loc[models, 'Recall'].values
ax3.bar(x - width/2, test_prec, width, label='Precision', color='#9467bd', alpha=0.8)
ax3.bar(x + width/2, test_rec, width, label='Recall', color='#ff7f0e', alpha=0.8)
ax3.set_ylabel('Score')
ax3.set_title('Precision vs Recall (Test Set)')
ax3.set_xticks(x)
ax3.set_xticklabels(models, rotation=45, ha='right')
ax3.legend()
ax3.set_ylim([0, 1.1])
ax3.grid(axis='y', alpha=0.3)

# 图4: F1 Score & G-Mean
ax4 = axes[1, 1]
test_f1 = df_test_results.set_index('Model').loc[models, 'F1 Score'].values
test_gmean = df_test_results.set_index('Model').loc[models, 'G-Mean'].values
ax4.bar(x - width/2, test_f1, width, label='F1 Score', color='#1f77b4', alpha=0.8)
ax4.bar(x + width/2, test_gmean, width, label='G-Mean', color='#17becf', alpha=0.8)
ax4.set_ylabel('Score')
ax4.set_title('F1 Score vs G-Mean (Test Set)')
ax4.set_xticks(x)
ax4.set_xticklabels(models, rotation=45, ha='right')
ax4.legend()
ax4.set_ylim([0, 1.1])
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'evaluation_metrics_comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

print("\n评估指标对比图已保存!")